In [ ]:
import numpy as np
from IPython import display
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import random
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from time import sleep
import plotly.graph_objects as go


In [ ]:
# Sphere
def Sphere_f(x):
    f = 0
    for i in x:
        f += i**2
    return f

# Michalewicz
def Michalewicz_f(x,m=10):
    d = len(x)
    j = np.arange(1, d+1)
    return -np.sum(np.sin(x) * np.sin(j*x**2/np.pi)**(2*m))

# Rosenbrock
def Rosenbrock_f(x):
    f = 0
    for i in range(len(x) - 1):
        f += (100*((x[i + 1] - x[i]**2)**2) + (1 -x[i])**2)
    return f

# Griewank
def Griewank_f(x):
    f1 = 0
    for i in x:
        f1 += (i*i)/4000
    f2 = 1
    for i in range(len(x)):
        f2 *= np.cos(x[i]/math.sqrt(i+1))
    return f1 - f2 + 1

# Ackley
def Ackley_f(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    f1 = 0
    f2 = 0
    for i in range(len(x)):
        f1 += x[i] ** 2
        f2 += np.cos(c * x[i])
    p1 = -a * np.exp(-b * np.sqrt(f1 / len(x)))
    p2 = -np.exp(f2 / len(x))

    return p1 + p2 + a + np.exp(1)

# **DE**

In [ ]:
def DE(fobj, dimension, bounds, F_scale, cross_prob, popsize, max_evals):

    lower_bound, upper_bound = np.asarray(bounds).T

    eps = 0.00001

    diff = np.fabs(lower_bound - upper_bound)

    pop = lower_bound + diff * np.random.rand(popsize, dimension)

    fitness = np.asarray([fobj(ind) for ind in pop])
    num_eval = 1

    best_idx = np.argmin(fitness)
    best = pop[best_idx]

    results = []
    all_pops = []
    results.append((np.copy(best), fitness[best_idx], num_eval))
    all_pops.append(np.copy(pop))
    generation_count = 0

    while True:
        if num_eval > max_evals:
            break
        for i in range(popsize):

            idxes = [idx for idx in range(popsize) if idx != i]
            a, b, c = pop[np.random.choice(idxes, 3, replace=False)]
            mutant = np.clip(F_scale*(b - c) + a, lower_bound, upper_bound)

            cross_points = np.random.rand(dimension) < cross_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimension)] = True

            trial = np.where(cross_points, mutant, pop[i])

            f = fobj(trial)
            num_eval += 1

            if f < fitness[i]:
                pop[i] = trial
                fitness[i] = f
                if f < fitness[best_idx]:
                    best = trial
                    best_idx = i

        results.append((np.copy(best), fitness[best_idx], num_eval))
        all_pops.append(np.copy(pop))

        if fobj(best) < eps:
            num_eval += 1
            break

        generation_count += 1

    return results, all_pops, generation_count


In [ ]:
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép

**Sphere**

In [ ]:
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
all_results

[(array([0.00027769, 0.00115155]), 1.4031794164968645e-06, 401),
 (array([-0.00093747, -0.00108967]), 2.06623547040266e-06, 497),
 (array([-0.00165337, -0.00024592]), 2.7941206623562983e-06, 465),
 (array([-0.00200062, -0.00241737]), 9.8461580581474e-06, 433),
 (array([-0.00078199, -0.00088744]), 1.3990480052721253e-06, 465),
 (array([0.00022429, 0.00073307]), 5.877002174963966e-07, 497),
 (array([-0.00168996, -0.00065476]), 3.284685591484262e-06, 417),
 (array([-0.0022551 ,  0.00114165]), 6.388837894906685e-06, 385),
 (array([ 0.00028865, -0.0017416 ]), 3.116503060207896e-06, 561),
 (array([-0.00246936,  0.00146469]), 8.243047153954241e-06, 513)]

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")

Giá trị trung bình (mean) của Sphere (d=2, n=16) trong DE: 3.912951553072483e-06
Độ lệch chuẩn (std) của Sphere (d=2, n=16) trong DE: 2.990816533399097e-06


In [ ]:
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=2, n=32) trong DE: 5.260767161054845e-06
Độ lệch chuẩn (std) của Sphere (d=2, n=32) trong DE: 2.1597188666687205e-06


[(array([ 0.0022325, -0.0006378]), 5.3908285479320535e-06, 769),
 (array([-0.0012849 ,  0.00082067]), 2.3244647438957298e-06, 961),
 (array([-0.00164547,  0.0002836 ]), 2.78800200683324e-06, 833),
 (array([0.00219856, 0.00200769]), 8.864501695208196e-06, 865),
 (array([-0.00279756, -0.00046651]), 8.043992693494645e-06, 865),
 (array([-3.80172056e-06,  1.78333477e-03]), 3.180297346516073e-06, 737),
 (array([-0.00263972,  0.00054475]), 7.2648862366693866e-06, 1057),
 (array([-0.00073025, -0.0021666 ]), 5.227413404639015e-06, 1025),
 (array([ 0.00084696, -0.00172305]), 3.6862257564742544e-06, 1025),
 (array([-0.00174855,  0.00166723]), 5.837059178885853e-06, 897)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=2, n=64) trong DE: 6.2182087321480825e-06
Độ lệch chuẩn (std) của Sphere (d=2, n=64) trong DE: 2.0530311052165725e-06


[(array([-2.57733688e-03,  5.45592566e-06]), 6.6426951550602396e-06, 1409),
 (array([-0.00235972, -0.00172941]), 8.559130883781122e-06, 1089),
 (array([0.00124581, 0.00056885]), 1.8756232622793637e-06, 1985),
 (array([0.00226933, 0.00082881]), 5.836801863765127e-06, 1473),
 (array([-1.96613514e-03, -9.49369452e-05]), 3.874700423237455e-06, 1729),
 (array([0.00286001, 0.00096418]), 9.10929274552628e-06, 1665),
 (array([-0.00225451, -0.00105986]), 6.206114675013218e-06, 1793),
 (array([-0.0015319 , -0.00195897]), 6.184255145024453e-06, 1729),
 (array([-0.00240348,  0.00018937]), 5.812598266919456e-06, 1089),
 (array([-0.00020404, -0.00283536]), 8.080874900874115e-06, 1537)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=2, n=128) trong DE: 5.571252819370205e-06
Độ lệch chuẩn (std) của Sphere (d=2, n=128) trong DE: 2.6982159931966766e-06


[(array([ 0.00206237, -0.00192886]), 7.973887895840235e-06, 2433),
 (array([0.00154888, 0.00032936]), 2.5075178407302754e-06, 2689),
 (array([-0.00238897, -0.00099636]), 6.699901067603581e-06, 2049),
 (array([ 0.00031582, -0.00088513]), 8.83204818201434e-07, 3585),
 (array([-0.00230924, -0.00186857]), 8.824168745493331e-06, 2817),
 (array([0.00121097, 0.00198792]), 5.418287579132863e-06, 2817),
 (array([-0.00066327,  0.00240309]), 6.214759125779358e-06, 3329),
 (array([ 0.00140184, -0.00083938]), 2.6697341509198223e-06, 3073),
 (array([0.0027684 , 0.00132168]), 9.410861079629891e-06, 2817),
 (array([-0.00158263,  0.00161415]), 5.110205890371248e-06, 3201)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=2, n=256) trong DE: 4.006020332535119e-06
Độ lệch chuẩn (std) của Sphere (d=2, n=256) trong DE: 2.7865255355775226e-06


[(array([-0.00293813,  0.00109295]), 9.827137639601884e-06, 4097),
 (array([-0.0005575 , -0.00199285]), 4.282267984479294e-06, 5377),
 (array([0.00150162, 0.00102685]), 3.309293840116385e-06, 5889),
 (array([-0.00054771,  0.00247008]), 6.401296291380321e-06, 5121),
 (array([-0.00185921,  0.00156288]), 5.899270673428101e-06, 4609),
 (array([-0.00102031, -0.00067763]), 1.5002238331721455e-06, 5889),
 (array([-2.30494414e-03, -5.55513104e-05]), 5.315853442420668e-06, 5889),
 (array([1.65654495e-05, 1.06186914e-03]), 1.1278404807244191e-06, 5377),
 (array([-0.00101098,  0.0010651 ]), 2.156508281960556e-06, 4353),
 (array([-0.00044364, -0.00020903]), 2.405108580674172e-07, 5889)]

In [ ]:
all_results = []
d = 10
max_evals = 100000
f = Sphere_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=10, n=16) trong DE: 7.431244229936978e-06
Độ lệch chuẩn (std) của Sphere (d=10, n=16) trong DE: 2.101097714130447e-06


[(array([-1.39285205e-05, -2.02662736e-03,  8.46189245e-04,  6.86661644e-04,
         -6.13653793e-04,  1.17212137e-03, -3.70481739e-04, -2.47173283e-04,
          1.23550246e-03,  8.02182899e-06]),
  8.770274413435754e-06,
  4545),
 (array([-0.00145706, -0.00164786, -0.00044031, -0.00107683,  0.000367  ,
          0.00100935,  0.00011111, -0.00030489, -0.00028429, -0.00045775]),
  7.741032937119089e-06,
  4657),
 (array([ 0.00064061,  0.0007514 ,  0.00061059, -0.00024954, -0.00222063,
         -0.00068866, -0.00046933, -0.00067346,  0.00043794, -0.00038707]),
  7.830982221874511e-06,
  5265),
 (array([ 0.00048129, -0.00125816, -0.00020506,  0.00016228,  0.0006396 ,
          0.00060836, -0.00119784, -0.00039759,  0.00033606, -0.00046763]),
  4.58667444399269e-06,
  4177),
 (array([-0.00138687, -0.00031239,  0.00062847, -0.0010721 , -0.00104886,
         -0.00076163,  0.00131599, -0.00050228,  0.0002026 , -0.00063318]),
  7.671651886700907e-06,
  4401),
 (array([ 8.30495573e-04,  3.033

In [ ]:
all_results = []
d = 10
max_evals = 100000
f = Sphere_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=10, n=32) trong DE: 8.397669716143907e-06
Độ lệch chuẩn (std) của Sphere (d=10, n=32) trong DE: 9.693145823834325e-07


[(array([-1.62157679e-03, -1.07659646e-03, -1.10257485e-03,  1.29411782e-03,
          6.65778537e-04,  1.96231881e-04,  2.21830576e-05, -5.79459198e-04,
         -5.42031143e-04,  1.21392577e-03]),
  9.264430034554053e-06,
  10337),
 (array([ 0.00045846,  0.0014482 , -0.00030043, -0.00089349,  0.00025852,
          0.00174191,  0.00062551, -0.00061002,  0.00054747, -0.00044202]),
  7.5556205062376535e-06,
  10689),
 (array([-7.68840766e-04, -2.20854012e-04, -9.88644726e-06,  5.30515610e-04,
         -7.14123718e-04, -1.71596726e-04,  1.65480261e-04, -1.97392558e-03,
          2.38130381e-04,  1.69982594e-03]),
  8.330735490877008e-06,
  10689),
 (array([-1.18820382e-04,  7.70890745e-04,  1.07397458e-03,  2.46895313e-05,
         -9.27477972e-04,  1.05632826e-03, -5.10515649e-04, -1.12027435e-03,
          1.72559343e-03, -9.28537228e-04]),
  9.093961509944125e-06,
  10817),
 (array([-6.15312538e-04, -1.70623109e-03, -2.38063679e-04,  5.25632529e-04,
          5.84931943e-04,  9.828416

In [ ]:
all_results = []
d = 10
max_evals = 100000
f = Sphere_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=10, n=64) trong DE: 7.945020494217491e-06
Độ lệch chuẩn (std) của Sphere (d=10, n=64) trong DE: 1.3957781614779357e-06


[(array([ 0.00090163,  0.00092253,  0.00038501, -0.00063023,  0.00144899,
         -0.00029294,  0.00207828,  0.00032204,  0.00050441,  0.00058262]),
  9.411661089995446e-06,
  23041),
 (array([ 6.72374278e-04, -4.75686086e-05,  7.59335850e-04,  4.71468358e-04,
         -1.57761838e-04, -1.25526830e-03, -2.23944318e-04, -1.64778111e-03,
         -5.03265973e-04,  1.25384190e-03]),
  7.444540401911244e-06,
  22145),
 (array([ 0.00089314,  0.00011358,  0.00032851,  0.00125068, -0.00034901,
          0.00094794, -0.00064641, -0.00159046,  0.00023377,  0.00041752]),
  6.679503166442964e-06,
  22145),
 (array([ 5.47609051e-04,  5.33710381e-05,  1.06832957e-03,  8.08512892e-04,
         -1.19665751e-03, -4.29179382e-04,  1.23256890e-03,  3.47209599e-04,
          3.43383623e-04, -5.62072636e-04]),
  5.787548022538899e-06,
  23233),
 (array([-0.00031267,  0.00068198,  0.00044884,  0.00115189, -0.00132296,
         -0.00063382, -0.0005033 ,  0.00230486, -0.00017143,  0.00020332]),
  9.87951073

In [ ]:
all_results = []
d = 10
max_evals = 100000
f = Sphere_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=10, n=128) trong DE: 8.760667631806214e-06
Độ lệch chuẩn (std) của Sphere (d=10, n=128) trong DE: 1.1701131583250958e-06


[(array([-0.00109693, -0.00122163, -0.00012568,  0.00028388,  0.0005634 ,
          0.00074981, -0.00072513,  0.0009822 ,  0.00125658, -0.00157435]),
  9.219757827434074e-06,
  44929),
 (array([-0.0012665 ,  0.00043312,  0.00120029, -0.00195002,  0.00044515,
         -0.00087183, -0.00027331,  0.00034527,  0.00024839,  0.0002414 ]),
  8.307004723580412e-06,
  44289),
 (array([-0.00028505, -0.0012629 ,  0.00073357,  0.00016108,  0.00103892,
         -0.00052911, -0.00014492, -0.00026146, -0.00141368, -0.00077627]),
  6.2900149692418935e-06,
  43777),
 (array([-1.03445035e-03, -4.98205152e-05,  1.33733695e-03,  1.91462440e-04,
          3.16585822e-04, -1.43339621e-03, -6.70432398e-04,  1.04732628e-03,
          5.40130836e-04,  1.42229780e-04]),
  6.910891441104764e-06,
  46977),
 (array([ 1.57816441e-03, -9.05835040e-04,  1.39056655e-04, -2.26671760e-04,
          1.94070304e-04,  6.89840447e-04, -3.32742837e-04, -2.31789646e-03,
          9.64872983e-05,  5.19869906e-04]),
  9.6583362

In [ ]:
all_results = []
d = 10
max_evals = 100000
f = Sphere_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Sphere (d=10, n=256) trong DE: 7.811761719456287e-06
Độ lệch chuẩn (std) của Sphere (d=10, n=256) trong DE: 1.5586514924447749e-06


[(array([-1.64665521e-05,  3.33668447e-04, -7.19932284e-04, -1.41581045e-03,
          1.60677919e-04, -5.92644602e-04, -5.55355833e-04,  2.22479993e-04,
          2.11095178e-04,  1.02331087e-03]),
  4.461116293043231e-06,
  89857),
 (array([-0.00070555,  0.0005084 ,  0.00115795,  0.00045814,  0.00030142,
         -0.00033873,  0.00021467, -0.00106401, -0.00148469,  0.00106571]),
  7.0308344182688845e-06,
  89857),
 (array([ 9.49521829e-04, -1.04407451e-03,  6.72324867e-06, -1.45164556e-03,
          1.47026482e-03, -1.22388612e-04,  6.17057944e-04, -6.51878646e-04,
         -1.16359214e-03, -5.04213010e-04]),
  8.68954464949206e-06,
  89345),
 (array([ 0.00075109,  0.00069476, -0.00018299,  0.00011667, -0.00106045,
         -0.00128355,  0.00115339,  0.00110373,  0.00072716, -0.00018933]),
  6.979113265281291e-06,
  84993),
 (array([ 1.12288012e-03, -1.36359450e-03, -6.48440090e-04,  1.34500017e-03,
          1.64068134e-03,  8.69605931e-05, -8.97628874e-04, -1.44633218e-04,
        

 **Griewank**

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =  Griewank_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=2, n=16) trong DE: 0.01440571574785906
Độ lệch chuẩn (std) của Griewank (d=2, n=16) trong DE: 0.016176330595915153


[(array([-3.14002264,  4.43844447]), 0.007396040334115006, 20001),
 (array([-0.00304102, -0.00310538]), 7.039460745428272e-06, 913),
 (array([-2.88280138, -4.43829176]), 0.040307327594977016, 20001),
 (array([-3.09303895,  4.43843925]), 0.008500088414865603, 20001),
 (array([ 0.00010232, -0.00089241]), 2.045346034762474e-07, 1313),
 (array([ 6.00000000e+00, -1.10227143e-08]), 0.04882971334963404, 20001),
 (array([-1.72795728e-01, -1.73305972e-09]), 0.014899536619344955, 20001),
 (array([-3.2765933 ,  4.43840377]), 0.016712599173324127, 20001),
 (array([-3.14002263, -4.43844448]), 0.007396040334114895, 20001),
 (array([ 0.00221059, -0.00494651]), 8.567662866054704e-06, 833)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =  Griewank_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=2, n=32) trong DE: 0.0029606863393465297
Độ lệch chuẩn (std) của Griewank (d=2, n=32) trong DE: 0.0036214515966385232


[(array([ 0.0021371 , -0.00384196]), 5.97858783468741e-06, 2785),
 (array([-3.14002263,  4.43844447]), 0.007396040334114895, 20001),
 (array([-0.00024013, -0.00270174]), 1.8555230620886931e-06, 1697),
 (array([-0.00048907,  0.00306119]), 2.4647122579546377e-06, 2497),
 (array([0.00307387, 0.00142173]), 5.2325287041021795e-06, 1793),
 (array([ 3.14002263, -4.43844447]), 0.007396040334114895, 20001),
 (array([-3.14002263,  4.43844448]), 0.007396040334114895, 20001),
 (array([ 0.00051719, -0.00439507]), 4.967800318556037e-06, 2017),
 (array([-3.14002264,  4.43844449]), 0.007396040334114895, 20001),
 (array([ 0.00042432, -0.00290567]), 2.2029048283300767e-06, 1729)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =  Griewank_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=2, n=64) trong DE: 4.556568605551536e-06
Độ lệch chuẩn (std) của Griewank (d=2, n=64) trong DE: 2.2198679259838094e-06


[(array([-0.00226416, -0.00100993]), 2.8197398328488887e-06, 4417),
 (array([0.00076077, 0.00055293]), 3.660368933511293e-07, 4033),
 (array([0.00238768, 0.00042556]), 2.8972450037967334e-06, 5761),
 (array([-3.38645541e-03,  5.96483923e-05]), 5.737792032722844e-06, 4929),
 (array([ 0.00172793, -0.00199886]), 2.493476141784967e-06, 4673),
 (array([0.00271623, 0.00270207]), 5.5179098259472426e-06, 4609),
 (array([-0.00307193, -0.0014984 ]), 5.282596947875362e-06, 3969),
 (array([-0.00344997, -0.00278771]), 7.898865975852765e-06, 6337),
 (array([-0.00314851,  0.00300203]), 7.214307185865465e-06, 3713),
 (array([-0.00324269,  0.0005567 ]), 5.3377162154699676e-06, 5441)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =  Griewank_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=2, n=128) trong DE: 3.6520164713893523e-06
Độ lệch chuẩn (std) của Griewank (d=2, n=128) trong DE: 1.882872952634642e-06


[(array([ 0.00016845, -0.00145714]), 5.455408478294288e-07, 11649),
 (array([-0.00140144, -0.00228873]), 2.2933857795059964e-06, 7041),
 (array([ 0.00270553, -0.00392536]), 7.5177215413946286e-06, 8577),
 (array([-0.00310739, -0.00061559]), 4.925168902158994e-06, 7041),
 (array([-0.00116976,  0.00210601]), 1.7944419076210139e-06, 10241),
 (array([ 0.00267609, -0.00183635]), 4.426396165158053e-06, 8193),
 (array([-0.00049325,  0.0041503 ]), 4.432262858511393e-06, 9345),
 (array([-0.00143189, -0.00215858]), 2.1916944961120066e-06, 9985),
 (array([0.00207212, 0.00282173]), 4.140444005096278e-06, 8321),
 (array([0.00271695, 0.00149639]), 4.253108210505729e-06, 10241)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =  Griewank_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=2, n=256) trong DE: 6.3540907821790785e-06
Độ lệch chuẩn (std) của Griewank (d=2, n=256) trong DE: 2.524154645896597e-06


[(array([ 0.00307701, -0.0040496 ]), 8.840233230111849e-06, 8961),
 (array([-0.00153737,  0.0049139 ]), 7.224963705865228e-06, 7681),
 (array([-0.00218207,  0.00468706]), 7.879516884412574e-06, 17665),
 (array([-0.00348336,  0.00065528]), 6.177372060323094e-06, 13825),
 (array([-0.00153969,  0.00305232]), 3.5174078751865068e-06, 18945),
 (array([-0.00170815,  0.00555619]), 9.18512714820352e-06, 17665),
 (array([0.00068177, 0.0048181 ]), 6.041849872628902e-06, 17153),
 (array([-0.00124905,  0.00242056]), 2.2466892045658327e-06, 19969),
 (array([-0.00356483,  0.00353306]), 9.480905300618403e-06, 14337),
 (array([-0.00193561,  0.00207031]), 2.946842539874872e-06, 16641)]

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =  Griewank_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=10, n=16) trong DE: 0.06508534157632337
Độ lệch chuẩn (std) của Griewank (d=10, n=16) trong DE: 0.045522072221158644


[(array([-2.40000000e-01, -9.36838211e-09,  1.45014461e-08, -1.88654626e-08,
          1.97815614e-08,  2.35720340e-08,  1.45903900e-08, -1.56680174e-08,
          8.90469293e-09, -1.14175288e-08]),
  0.028676425147969753,
  100001),
 (array([-6.00000000e+00, -2.18916887e-09, -5.43282497e+00, -6.00000000e+00,
         -9.50148369e-09,  4.16512333e-09, -5.38652599e-09,  1.91572934e-08,
         -2.77439033e-08, -2.59901286e-08]),
  0.07482916165095066,
  100001),
 (array([-6.03416038e-09,  4.43839967e+00, -8.72239577e-09, -6.00000000e+00,
          1.24931587e-08,  1.89737366e-08,  5.19139996e-09, -8.69858072e-09,
          1.15945438e-08,  9.02679181e-09]),
  0.023937325939459853,
  100001),
 (array([ 6.00000000e+00, -4.43821387e+00,  1.68559576e-08,  6.00000000e+00,
          2.24456101e-09,  2.16987077e-08,  1.29098400e-08, -9.19149391e-09,
         -2.36880670e-08,  2.93969269e-08]),
  0.0723682369481411,
  100001),
 (array([-1.35433476e-01, -4.43835827e+00,  1.40586445e-09, -6.0000

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =  Griewank_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=10, n=32) trong DE: 0.05425161405078639
Độ lệch chuẩn (std) của Griewank (d=10, n=32) trong DE: 0.018612792012084402


[(array([-6.00000000e+00,  1.26001376e-08,  5.43282498e+00, -6.00000000e+00,
          1.05221482e-08,  2.34728645e-08,  4.62803292e-10, -6.58920501e-10,
         -2.64757059e-08,  3.27933713e-08]),
  0.07482916165095066,
  100001),
 (array([-2.96203144e+00, -4.43832643e+00, -5.43303157e+00, -6.00000000e+00,
          2.10291680e-08, -5.01819924e-09,  1.98400780e-08,  2.76480967e-09,
          1.68615203e-08, -3.00485634e-08]),
  0.0494384217435615,
  100001),
 (array([ 3.14000675e+00, -4.43839961e+00,  5.43316588e+00, -6.00000000e+00,
         -4.51218593e-09,  7.07417570e-09,  1.80055566e-08, -2.07942114e-08,
         -2.43586763e-09, -2.99569319e-08]),
  0.03379448612330149,
  100001),
 (array([-2.52780897e-01, -4.43825267e+00, -2.99639730e-09, -6.00000000e+00,
          1.85667758e-09,  3.09785307e-09,  4.36437922e-09,  1.65698235e-08,
          9.77182009e-10, -1.81755068e-08]),
  0.055414434240585564,
  100001),
 (array([-3.36099723e+00, -4.43828961e+00, -5.43296400e+00,  6.00000

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =  Griewank_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=10, n=64) trong DE: 0.04496948446549633
Độ lệch chuẩn (std) của Griewank (d=10, n=64) trong DE: 0.024762334897446642


[(array([-6.98999994e-09,  5.09728830e-11,  7.11869222e-09,  1.13042937e-01,
         -7.39169147e-09, -2.27282105e-09, -1.46841545e-08,  3.93667107e-09,
         -3.15739235e-08, -1.20871820e-08]),
  0.0016001076658218594,
  100033),
 (array([-6.00000000e+00, -7.48427026e-09,  5.43282498e+00,  6.00000000e+00,
         -5.39078516e-09, -2.12171652e-09, -1.20758822e-09, -6.51302992e-09,
         -2.36197791e-08,  6.28540038e-09]),
  0.07482916165095066,
  100033),
 (array([ 3.18149947e+00,  4.43839605e+00,  5.43315935e+00,  6.00000000e+00,
         -4.23900785e-09,  1.56254736e-08,  4.79740023e-09,  9.03031856e-09,
         -2.84861298e-08,  1.96849741e-09]),
  0.034647006106900635,
  100033),
 (array([ 6.00000000e+00, -4.43826050e+00, -5.43291060e+00,  2.77412351e-11,
          1.80067902e-08,  2.06781064e-08,  2.42594049e-08,  1.81333283e-08,
          1.33231389e-08,  5.50197433e-09]),
  0.06115003874738112,
  100033),
 (array([-6.00000000e+00, -4.43826051e+00, -5.43291059e+00, -1.14

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =  Griewank_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=10, n=128) trong DE: 0.01549791789314603
Độ lệch chuẩn (std) của Griewank (d=10, n=128) trong DE: 0.014862093482124262


[(array([ 2.40000000e-01, -4.43825734e+00,  6.59331256e-06,  6.00000000e+00,
         -8.62513407e-05, -1.17354428e-04,  2.70731470e-05, -2.16681942e-05,
         -2.05621814e-06, -6.59225068e-05]),
  0.052326771132299,
  100097),
 (array([-0.00250483, -0.00354703,  0.00358774,  0.00347528,  0.00128405,
          0.00420061,  0.00946768, -0.00059805,  0.00034671, -0.00764027]),
  2.097582883686666e-05,
  100097),
 (array([ 3.14070175e+00,  4.01325999e-03, -5.43030936e+00,  3.90668084e-03,
          2.72618083e-03,  4.31631196e-04,  2.27957076e-03, -2.09465926e-03,
          4.85900982e-03, -2.88211956e-03]),
  0.009868042274272848,
  100097),
 (array([-0.00144227,  0.03444956, -0.00396744, -0.11826928, -0.03016067,
          0.0265257 , -0.0043755 , -0.07297868, -0.0507985 ,  0.00675792]),
  0.002681994832953616,
  100097),
 (array([ 3.08855130e+00, -1.32969785e-05, -4.54490062e-06,  6.00000000e+00,
         -3.40239657e-05, -9.66817448e-06,  4.99382312e-06,  5.38224908e-06,
          

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =  Griewank_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của Griewank (d=10, n=256) trong DE: 0.04400498224929463
Độ lệch chuẩn (std) của Griewank (d=10, n=256) trong DE: 0.00943185133602668


[(array([ 3.21343162e+00,  2.56700838e-04, -1.38122956e-01, -6.00000000e+00,
         -3.93162241e-01,  1.13755691e-01,  2.23270916e-01,  2.94764764e-01,
          1.18069223e-01,  4.58069883e-04]),
  0.053007180473601645,
  100097),
 (array([-3.24959940e+00, -4.41634988e+00, -5.57227641e+00,  6.00000000e+00,
         -8.88638666e-02,  7.38586644e-02,  1.22897294e-01, -8.42390628e-02,
          1.41271146e-01,  3.53089604e-03]),
  0.04684695997650701,
  100097),
 (array([ 0.07237846,  0.01623932, -5.47056469,  6.        ,  0.08373798,
          0.0271719 , -0.19631295,  0.1012397 ,  0.08628695,  0.16664145]),
  0.03517852570469482,
  100097),
 (array([-3.09984690e+00,  4.31085563e+00, -5.49251789e+00, -6.00000000e+00,
         -4.91447281e-03,  3.25417774e-02,  2.45563344e-01, -7.15222557e-02,
          8.97706748e-02,  3.16037826e-01]),
  0.049204300449740224,
  100097),
 (array([ 0.0429194 ,  0.01516198,  5.54391832, -6.        , -0.04999136,
         -0.19650289, -0.18281217, -0.249

**Rosenbrock**

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=2, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=2, n=16) trong DE: 4.462655181088748e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=16) trong DE: 3.3705796517303046e-06


[(array([1.00238378, 1.00473654]), 5.817099492383002e-06, 1585),
 (array([0.9975378 , 0.99505583]), 6.1291519476299706e-06, 1473),
 (array([1.00208024, 1.0043865 ]), 9.242196850628304e-06, 1793),
 (array([0.99968738, 0.99935587]), 1.3377984961777507e-07, 1105),
 (array([1.00025049, 1.00031209]), 3.6326482492177592e-06, 1537),
 (array([0.99939722, 0.99873556]), 7.142233033983073e-07, 1297),
 (array([0.99951059, 0.99909012]), 7.11634415449753e-07, 1617),
 (array([1.00267629, 1.00526983]), 7.970920732868845e-06, 1153),
 (array([0.99986199, 0.99959926]), 1.574813800044362e-06, 849),
 (array([0.99705042, 0.99411035]), 8.700083169649403e-06, 1713)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=2, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=2, n=32) trong DE: 3.1439200996882823e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=32) trong DE: 2.0276300369268095e-06


[(array([0.99877559, 0.99750257]), 1.7502496792182345e-06, 2849),
 (array([1.00174888, 1.00362885]), 4.697628792781313e-06, 2401),
 (array([0.99790671, 0.99568031]), 6.272338864102189e-06, 2625),
 (array([0.99930961, 0.99863374]), 4.963554505031475e-07, 3041),
 (array([0.9987363, 0.9972644]), 5.997996093704324e-06, 2337),
 (array([0.9988435 , 0.99760565]), 2.0211431267675698e-06, 2593),
 (array([0.99794036, 0.9958438 ]), 4.411475367241844e-06, 3233),
 (array([0.9994415 , 0.99884375]), 4.685133717188656e-07, 2689),
 (array([1.00186321, 1.0037361 ]), 3.475400538796629e-06, 2497),
 (array([1.00135945, 1.00272049]), 1.848099712048706e-06, 2945)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=2, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=2, n=64) trong DE: 3.967247683790893e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=64) trong DE: 2.477018607514163e-06


[(array([1.00082669, 1.00175327]), 1.6675200163647342e-06, 4865),
 (array([0.99690444, 0.99377402]), 9.779974412666144e-06, 5377),
 (array([0.99960148, 0.99937305]), 3.0465881693387623e-06, 5313),
 (array([0.99899036, 0.99782876]), 3.3594632504976514e-06, 4801),
 (array([0.99829428, 0.99642196]), 5.782986678393006e-06, 4737),
 (array([0.99913355, 0.99843231]), 3.455103801064332e-06, 5313),
 (array([0.99910071, 0.99840706]), 5.004666756359124e-06, 4097),
 (array([1.00131721, 1.00265545]), 1.772267439084392e-06, 4993),
 (array([1.00166922, 1.00319035]), 5.062922346423709e-06, 5633),
 (array([0.99925274, 0.99846331]), 7.409839677170713e-07, 5889)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=2, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=2, n=128) trong DE: 2.7766985664739693e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=128) trong DE: 2.6048629801644567e-06


[(array([0.99980354, 0.99957512]), 1.4101478350107042e-07, 10113),
 (array([0.99917552, 0.99834969]), 6.801810859910137e-07, 10625),
 (array([0.99714232, 0.99429122]), 8.166596974664403e-06, 9729),
 (array([0.9983835 , 0.99677492]), 2.6158977728626177e-06, 9217),
 (array([0.99955355, 0.99935794]), 6.481631014376693e-06, 11137),
 (array([0.99987187, 0.99981593]), 5.373442407747323e-07, 9473),
 (array([1.00090402, 1.00174683]), 1.2019374263653093e-06, 8961),
 (array([0.99912227, 0.9981161 ]), 2.4400713250929866e-06, 9985),
 (array([1.0005873 , 1.00125619]), 1.004972901712298e-06, 9729),
 (array([1.0020481, 1.0041554]), 4.497338139398569e-06, 9985)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=2, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=2, n=256) trong DE: 2.035032639049501e-05
Độ lệch chuẩn (std) của  Rosenbrock (d=2, n=256) trong DE: 1.9771872851874553e-05


[(array([0.9994502 , 0.99869254]), 4.635556219147838e-06, 17409),
 (array([1.00030382, 1.00145615]), 7.207444199866282e-05, 20225),
 (array([0.99734227, 0.99462099]), 7.562163588236864e-06, 19969),
 (array([1.00371766, 1.00776278]), 2.3657553418184865e-05, 20225),
 (array([0.99662737, 0.9931494 ]), 1.273674036561636e-05, 20225),
 (array([1.0026583 , 1.00480188]), 3.4293589713986626e-05, 20225),
 (array([0.9975341, 0.9950107]), 6.484866517127059e-06, 19969),
 (array([0.99849656, 0.99708044]), 2.9839290823626247e-06, 19969),
 (array([1.00238382, 1.00448772]), 1.3840272130683979e-05, 20225),
 (array([0.99699497, 0.99359643]), 2.5234150870941072e-05, 20225)]

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

In [ ]:
print(f"Giá trị trung bình (mean) của  Rosenbrock (d=10, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=10, n=16) trong DE: 6.96115348830294e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=16) trong DE: 2.4506561172053505e-06


[(array([0.99997494, 0.99996169, 1.00006499, 1.00012124, 1.00006687,
         1.00011689, 1.00015145, 1.00044671, 1.00084458, 1.00170923]),
  9.137857239815075e-06,
  32625),
 (array([0.99997868, 0.99997988, 0.99987916, 0.99998459, 1.00003263,
         1.00017096, 1.00027291, 1.00055985, 1.00111999, 1.00230356]),
  9.911659731549668e-06,
  29745),
 (array([0.99997755, 0.99996409, 0.99998424, 0.99995079, 1.00009191,
         1.00010119, 1.00026597, 1.00052595, 1.00100997, 1.0020901 ]),
  7.113969817338961e-06,
  32433),
 (array([0.9999693 , 1.00004759, 0.99999493, 0.99997323, 1.00003485,
         1.00009802, 1.0001703 , 1.00035871, 1.00064968, 1.00129992]),
  4.236126944884576e-06,
  34641),
 (array([1.00005727, 1.00006662, 0.99999231, 0.99999613, 0.99996768,
         0.99992917, 0.99989978, 0.99978077, 0.99961962, 0.9993368 ]),
  4.00209014559045e-06,
  33649),
 (array([0.99990273, 0.99998684, 0.99997565, 0.99995212, 0.99986561,
         0.99991584, 0.99985026, 0.99967926, 0.99925091, 

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Rosenbrock (d=10, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=10, n=32) trong DE: 8.48380444990133e-06
Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=32) trong DE: 1.034945168674524e-06


[(array([1.00000697, 1.00001381, 1.00003788, 0.99989964, 0.99996163,
         0.99999024, 1.00000508, 0.99998662, 0.99991969, 0.9998631 ]),
  6.674897834975188e-06,
  65761),
 (array([1.00001405, 0.99993132, 0.99997708, 1.00010037, 1.0001253 ,
         1.00013743, 1.00032452, 1.00064553, 1.00123264, 1.00242983]),
  9.05629971794932e-06,
  69025),
 (array([1.00000597, 1.00003865, 1.000031  , 0.99997045, 0.99992751,
         0.99973659, 0.99950851, 0.99910234, 0.9981643 , 0.99623453]),
  9.006686623777134e-06,
  67553),
 (array([1.00003248, 0.99999633, 0.9999987 , 0.99997958, 1.00006452,
         1.0000527 , 1.00020615, 1.00052468, 1.00116915, 1.00222454]),
  8.92356070256286e-06,
  74209),
 (array([0.99996524, 1.00006206, 0.99999098, 1.00006221, 1.00002693,
         1.00007638, 1.00012078, 1.00017586, 1.00022523, 1.00036836]),
  8.070720638449871e-06,
  75873),
 (array([0.99996954, 0.99997068, 0.99994209, 0.99985664, 0.99993838,
         0.99990517, 0.99995767, 0.99986547, 0.99976999, 0

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Rosenbrock (d=10, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=10, n=64) trong DE: 0.0032897777175864666
Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=64) trong DE: 0.0014302325878774974


[(array([1.00085716, 0.99994074, 0.99800949, 0.99779422, 0.99728281,
         0.99455097, 0.99265348, 0.98635618, 0.97573754, 0.95409705]),
  0.004698755834180345,
  100033),
 (array([1.00090975, 1.0003993 , 1.00044122, 1.00221077, 1.00065438,
         1.00024198, 1.00070023, 0.99960245, 1.00199485, 1.00467944]),
  0.003092488932068416,
  100033),
 (array([0.99764694, 0.99956696, 0.99941021, 1.00026896, 0.99849852,
         0.99697801, 0.99629436, 0.99180994, 0.98592608, 0.97285571]),
  0.003922515299599205,
  100033),
 (array([0.99934225, 0.9998774 , 0.99969629, 0.99898801, 0.99912114,
         0.99917939, 0.99921708, 0.99918017, 0.99891517, 0.99817225]),
  0.0005543890431456548,
  100033),
 (array([0.99985732, 0.99925696, 0.99908978, 0.99976231, 0.99952244,
         0.99786089, 0.99723512, 0.99427218, 0.98811349, 0.97761185]),
  0.0010405613984834565,
  100033),
 (array([1.00095737, 0.99986824, 1.00061929, 1.00039395, 1.00017905,
         0.99754359, 0.99507139, 0.99223083, 0.9819419

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Rosenbrock (d=10, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=10, n=128) trong DE: 1.4523203808170428
Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=128) trong DE: 0.36347102451991825


[(array([0.99206427, 0.99850635, 0.97806144, 0.94961416, 0.90989556,
         0.81521478, 0.626095  , 0.42782522, 0.17218019, 0.01748813]),
  1.5410848361416702,
  100097),
 (array([1.01245056, 0.99771857, 0.98768281, 0.96063301, 0.90618788,
         0.83241385, 0.73995193, 0.5121462 , 0.23125441, 0.04741305]),
  1.5250138030995792,
  100097),
 (array([0.99007902, 0.96920927, 0.96522789, 0.9405412 , 0.89633424,
         0.83955351, 0.73202071, 0.54410407, 0.29514295, 0.0741635 ]),
  1.1475196318998098,
  100097),
 (array([0.99642719, 0.98477726, 0.95017068, 0.93686149, 0.84898715,
         0.75159446, 0.54285286, 0.32224707, 0.12176115, 0.04185597]),
  2.0988840703117893,
  100097),
 (array([0.99074276, 0.98530418, 0.96059088, 0.9481998 , 0.88950905,
         0.80932884, 0.66193009, 0.43315154, 0.1855779 , 0.01679914]),
  1.3094600730307708,
  100097),
 (array([0.9946723 , 0.9914901 , 0.99156881, 0.9833272 , 0.9350235 ,
         0.8556903 , 0.74082428, 0.57892739, 0.34279854, 0.0862876

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Rosenbrock (d=10, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Rosenbrock (d=10, n=256) trong DE: 6.294138404511484
Độ lệch chuẩn (std) của  Rosenbrock (d=10, n=256) trong DE: 0.17008042502475207


[(array([ 0.84104514,  0.68265721,  0.48327479,  0.25697487,  0.07630366,
         -0.04445863,  0.02632778, -0.01880888,  0.02276687, -0.00770438]),
  6.393481532798731,
  100097),
 (array([0.82018925, 0.69407714, 0.50412181, 0.21913766, 0.0810865 ,
         0.00676118, 0.01773798, 0.05751525, 0.03712329, 0.00265795]),
  6.39331531865317,
  100097),
 (array([ 0.87453288,  0.77508862,  0.60522696,  0.31456701,  0.09953394,
          0.01080029,  0.03818362, -0.04921559,  0.00637808,  0.0203204 ]),
  6.219260693575172,
  100097),
 (array([ 0.86413192,  0.78629997,  0.62166415,  0.40064554,  0.16170994,
          0.06798634,  0.05880475, -0.04413279,  0.02804366, -0.0406421 ]),
  6.171482379448802,
  100097),
 (array([ 0.89472236,  0.80706563,  0.71074908,  0.46225796,  0.15673497,
          0.02554988, -0.0171749 , -0.00144199, -0.01629915,  0.00565973]),
  6.079283952916576,
  100097),
 (array([ 0.77866197,  0.61981855,  0.38250033,  0.16910475,  0.03132378,
          0.03852946, -0.00

**Michalewicz**

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=2, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=2, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=2, n=16) trong DE: -0.8569867589843101
Độ lệch chuẩn (std) của  Michalewicz (d=2, n=16) trong DE: 0.36624276485051765


[(array([ 6.        , -4.71521446]), -0.967102473001942, 17),
 (array([2.21608352, 1.10728205]), -0.7992619106086537, 17),
 (array([ 2.79320194, -4.72890501]), -0.905902681866326, 17),
 (array([-4.88693184,  2.93110953]), -0.5184680662152902, 17),
 (array([-3.73649362,  1.5713427 ]), -1.2703633190843462, 17),
 (array([ 2.15483766, -2.37453315]), -0.7653378515571447, 17),
 (array([-4.87555134,  3.88428768]), -0.4269002080189726, 17),
 (array([-3.90595944, -3.95786278]), -0.5622121415881381, 17),
 (array([-5.02111114,  1.58393856]), -1.7028613707569114, 17),
 (array([-3.84689569, -6.        ]), -0.6514575671453773, 17)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=2, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=2, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=2, n=32) trong DE: -0.9701374701040095
Độ lệch chuẩn (std) của  Michalewicz (d=2, n=32) trong DE: 0.166176544973079


[(array([-5.45426167, -4.13230128]), -0.7156519774022881, 33),
 (array([-3.88008337, -4.76275702]), -1.0226686814406272, 33),
 (array([2.15349919, 1.73894878]), -1.036807024080588, 33),
 (array([-2.94266402,  1.55825668]), -0.993701046234037, 33),
 (array([ 4.62771461, -4.71287702]), -0.999912789873073, 33),
 (array([-4.96376955,  1.28061742]), -1.019391246332667, 33),
 (array([ 2.39351882, -4.71944744]), -1.338737856209883, 33),
 (array([2.73828824, 1.49924084]), -0.8199283759237812, 33),
 (array([ 2.15550033, -3.20980607]), -0.7662915651594506, 33),
 (array([1.66285053, 1.5913449 ]), -0.9882841383836999, 33)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=2, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=2, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=2, n=64) trong DE: -1.0660437202501556
Độ lệch chuẩn (std) của  Michalewicz (d=2, n=64) trong DE: 0.150720626609673


[(array([-4.9418026, -1.8229182]), -0.8702929317896595, 65),
 (array([ 2.37246181, -4.73718591]), -1.2249504252091672, 65),
 (array([ 3.26324873, -4.70623663]), -0.9864617234978126, 65),
 (array([-3.05182284, -4.73067382]), -0.885821770759177, 65),
 (array([-4.45819846, -4.7171527 ]), -0.9918432274249567, 65),
 (array([ 2.24505148, -4.65739087]), -1.1060740648856402, 65),
 (array([4.1361415 , 1.55857622]), -0.9918073349971288, 65),
 (array([-6.        ,  1.56279374]), -1.0274443928828245, 65),
 (array([-4.91569136, -5.64086344]), -1.2095743130938907, 65),
 (array([2.21667747, 1.68438084]), -1.3661670179612986, 65)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=2, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=2, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=2, n=128) trong DE: -1.4018117457142858
Độ lệch chuẩn (std) của  Michalewicz (d=2, n=128) trong DE: 0.29129588043704635


[(array([-3.89149539,  2.73023482]), -1.0018759499944463, 129),
 (array([2.20040501, 1.60308241]), -1.759009307245722, 129),
 (array([-4.80587776, -4.6981155 ]), -1.0006992147093845, 129),
 (array([2.14161735, 1.52806974]), -1.6744442827565278, 129),
 (array([ 2.14441726, -4.68957728]), -1.5780672983874076, 129),
 (array([-6.        ,  1.57432991]), -1.0295151228942987, 129),
 (array([ 1.99366465, -4.69980319]), -1.2976560098025143, 129),
 (array([-5.01364139, -4.71396095]), -1.7673257380259357, 129),
 (array([2.18251705, 1.47730676]), -1.5097495001947512, 129),
 (array([-3.90096362, -4.16804811]), -1.3997750331318701, 129)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=2, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=2, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=2, n=256) trong DE: -1.3018540657405908
Độ lệch chuẩn (std) của  Michalewicz (d=2, n=256) trong DE: 0.16393651854175123


[(array([ 2.256711  , -5.64727278]), -1.271710972619975, 257),
 (array([-6.        ,  1.55385753]), -1.01858857689257, 257),
 (array([-3.79175839, -5.19104418]), -1.2641168229981377, 257),
 (array([-3.84784687,  1.6385289 ]), -1.4723022049073156, 257),
 (array([-3.78974839, -4.15274872]), -1.3399181987202435, 257),
 (array([-3.82303241, -4.16174989]), -1.451699214775688, 257),
 (array([2.29323443, 1.61852979]), -1.5834936811116271, 257),
 (array([-4.95721453, -3.53098493]), -1.3145392358581232, 257),
 (array([1.93721839, 1.5800841 ]), -1.2154321334983722, 257),
 (array([2.2927245 , 2.71734317]), -1.0867396160238567, 257)]

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=10, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=10, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=10, n=16) trong DE: -1.5510082674554408
Độ lệch chuẩn (std) của  Michalewicz (d=10, n=16) trong DE: 0.3849647783375788


[(array([-6.        ,  2.42082074, -6.        ,  1.13172464,  2.38264944,
         -0.59050854,  2.53467601,  0.68511746,  3.45798598,  4.48221843]),
  -1.351516459272564,
  17),
 (array([ 5.28472779, -4.08832757, -5.67197919, -5.8174717 , -5.81555637,
         -3.64380144, -3.8470888 ,  3.29167449,  2.51223539,  2.1394884 ]),
  -0.9967450569855276,
  17),
 (array([-5.27437966, -2.63301116, -5.60041452, -4.27713086, -4.08433865,
          4.18621285,  1.8902392 , -5.76064713,  3.45984683,  5.8681796 ]),
  -2.2098581224119886,
  17),
 (array([ 2.08922669,  5.25975932,  0.74672458,  3.54704256, -5.47587553,
          2.07578905,  1.85848438, -4.36976312, -3.28432503,  4.94723491]),
  -2.198340836063492,
  17),
 (array([-5.45759579,  1.44996289, -5.57531923, -4.9759743 , -1.81119133,
          1.83702035, -1.0497291 ,  1.45297589, -5.18686767,  3.13432812]),
  -1.816805097858666,
  17),
 (array([-6.        ,  3.97000455, -5.04654987, -0.44464937,  6.        ,
         -4.14880754,  0.1006

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=10, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=10, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=10, n=32) trong DE: -1.852725966478678
Độ lệch chuẩn (std) của  Michalewicz (d=10, n=32) trong DE: 0.37500285093808594


[(array([-6.        , -5.45930712,  1.34077676,  1.90164072,  3.05421192,
         -2.00798001, -2.74538134, -4.27369479,  2.23101056,  3.98577619]),
  -1.5899381671114334,
  33),
 (array([-0.6803027 ,  1.68913182, -1.96922944,  1.83507483,  1.74250149,
         -3.01523979,  5.73612811, -1.24912387,  2.70104887, -4.13121826]),
  -1.43763257827723,
  33),
 (array([ 6.        ,  0.32834105,  6.        ,  5.19457868,  6.        ,
          3.17199248, -3.43325209,  1.66415306,  1.26695788,  2.11873108]),
  -1.5246866154642156,
  33),
 (array([-5.92253851, -5.23001222, -1.86486611, -5.11569161,  0.20399302,
          4.01496584,  4.30523818,  0.73349613, -2.14354404,  2.61613247]),
  -1.7291938322167755,
  33),
 (array([ 3.24216973,  1.11248099,  1.01828786, -5.03891781, -3.8966634 ,
          0.23724919,  0.85161733,  1.75426394,  1.30646311, -4.37360501]),
  -2.694439369604371,
  33),
 (array([-1.18048395, -5.74460264, -5.9297556 ,  1.01076225,  0.99195821,
         -0.27509916, -2.0031

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=10, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=10, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=10, n=64) trong DE: -2.1823259917081197
Độ lệch chuẩn (std) của  Michalewicz (d=10, n=64) trong DE: 0.5417266871471551


[(array([-2.41677133, -4.71511327, -0.68863551, -2.02513804, -4.51922074,
         -4.71130664,  3.51031236,  2.48260365, -5.2040425 , -2.593144  ]),
  -1.7508753079787245,
  65),
 (array([ 2.43382536, -0.92056651, -4.94574135,  1.12377118, -0.30809771,
         -0.60115907, -0.9802909 ,  1.68659374,  0.31456382,  0.52039279]),
  -1.7606272960904956,
  65),
 (array([-4.99153009,  1.49572212, -5.31972361,  0.26567433,  3.38387423,
         -0.46344804, -4.81833572, -5.92399559,  0.65605352,  1.92584775]),
  -3.340806457762585,
  65),
 (array([-6.        ,  2.18332689, -4.44706531,  0.25103189,  6.        ,
          2.53959214,  0.25576711, -4.90146275, -0.54460221, -4.71453106]),
  -1.9035327351750833,
  65),
 (array([-5.04644392, -4.69474144, -0.56171219,  5.34995481,  3.28631977,
         -4.08371452,  0.88637769, -4.35462284, -0.83712441,  3.63148134]),
  -1.5236375094882995,
  65),
 (array([ 2.86070168,  1.56490037,  5.94315437,  2.01603633,  1.8379829 ,
         -4.35651737,  1.45

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=10, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=10, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=10, n=128) trong DE: -2.431819017582115
Độ lệch chuẩn (std) của  Michalewicz (d=10, n=128) trong DE: 0.22777201441491762


[(array([-3.86992795,  3.84722385,  3.53709543, -0.77036111, -5.52380033,
         -3.34499587, -4.55953589, -4.51886269, -4.45723304,  3.05748902]),
  -2.0526710936528425,
  129),
 (array([-3.28222261, -1.16674991,  4.45925634, -3.72909203, -6.        ,
         -5.15638307, -4.5147749 ,  1.74295687,  6.        ,  1.56462832]),
  -2.7148081362743617,
  129),
 (array([ 2.27407871, -0.40116363,  3.10797735, -3.03258595,  1.37927315,
         -4.52927178, -0.39203303, -1.52702849,  5.05556489, -4.60050442]),
  -2.337923701837559,
  129),
 (array([ 2.09964549,  2.61027103, -0.05676994, -5.28908567,  2.21466186,
         -5.90080561, -4.4460862 ,  3.79267856,  2.21353045,  5.97527564]),
  -2.2876582264397856,
  129),
 (array([ 2.2675054 ,  0.55033081, -2.07273689,  2.95267047, -4.55285871,
         -4.80366822, -4.01750871,  3.90323748, -4.07840995, -3.3847843 ]),
  -2.4652402886797367,
  129),
 (array([ 2.49175295, -5.19622206,  1.36050925, -5.31797667,  2.89256004,
          2.50876248, 

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Michalewicz (d=10, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Michalewicz (d=10, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Michalewicz (d=10, n=256) trong DE: -2.4955498223402492
Độ lệch chuẩn (std) của  Michalewicz (d=10, n=256) trong DE: 0.39604304032664867


[(array([ 2.29044102, -4.27020117,  2.83138813,  1.93085684, -0.49417847,
         -2.66623851,  1.2259287 , -3.07803534, -4.64275737, -4.66984246]),
  -1.8907013994441808,
  257),
 (array([ 0.04895751, -2.51396217, -5.5855878 ,  5.5177875 , -3.82889661,
         -4.53344334,  3.21531578,  1.41395047, -4.38643751,  1.33997532]),
  -2.922422371952017,
  257),
 (array([-6.        ,  2.73102821, -0.94948823, -1.37724747,  3.41312416,
          3.44565355,  0.77961466,  3.01295396,  2.22237674,  0.63683618]),
  -1.9381822814118106,
  257),
 (array([-5.11547065, -4.77910762, -2.65325926,  5.3894956 ,  0.94686704,
          4.66020547,  3.94520775,  5.41722801, -5.07560861,  2.3230897 ]),
  -2.5230161922691443,
  257),
 (array([-1.92466685, -2.04413989, -0.34610731, -0.63860908,  2.22008755,
          0.90268306,  0.82032263,  0.77756776,  4.90477171,  2.21245584]),
  -2.7746714576475306,
  257),
 (array([-2.59549011,  5.43970443, -4.25931736, -3.11397754, -4.7884262 ,
         -0.11960684, 

**Ackley**

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=2, n=16) trong DE: 5.617608431496279e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=16) trong DE: 1.346706300799061e-06


[(array([-6.48883906e-07,  1.16021459e-06]), 3.7599919662234527e-06, 913),
 (array([-1.52852590e-06,  1.82552919e-06]), 6.734509579775505e-06, 1073),
 (array([1.72866315e-06, 2.08267856e-07]), 4.924835967035079e-06, 913),
 (array([ 1.47812098e-06, -1.30137488e-06]), 5.570322813230888e-06, 993),
 (array([-2.94704617e-07,  1.97648001e-06]), 5.652237984765662e-06, 1057),
 (array([ 2.31278601e-06, -1.83265244e-06]), 8.346533615810614e-06, 913),
 (array([1.42075757e-06, 1.23461577e-06]), 5.323873577101068e-06, 1137),
 (array([1.15767113e-06, 6.39813786e-07]), 3.7412379794155015e-06, 865),
 (array([-2.26821366e-06,  9.25624356e-07]), 6.929271528566261e-06, 993),
 (array([ 1.68749579e-06, -7.23531042e-07]), 5.193269303038761e-06, 1057)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=2, n=32) trong DE: 4.903323686367145e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=32) trong DE: 1.6722253593790642e-06


[(array([-9.89099352e-07, -7.16285876e-07]), 3.454176923600727e-06, 2017),
 (array([-1.93028025e-06, -4.20278504e-08]), 5.461050218702468e-06, 1729),
 (array([-1.90474499e-06, -1.30414779e-06]), 6.529371515018312e-06, 1825),
 (array([-5.64728068e-08,  1.58027575e-06]), 4.472614510309114e-06, 1857),
 (array([-1.64998637e-06,  3.16763388e-07]), 4.752164440180451e-06, 1825),
 (array([7.77288797e-07, 5.22178295e-07]), 2.648567168694882e-06, 1953),
 (array([-1.08784187e-07,  2.35739435e-06]), 6.674961920527522e-06, 1921),
 (array([-7.54423049e-07,  4.80484458e-08]), 2.1381691657218482e-06, 2081),
 (array([1.17974705e-06, 1.51946363e-06]), 5.44110787048524e-06, 2017),
 (array([2.36326689e-06, 1.17176715e-06]), 7.461053130430884e-06, 2049)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=2, n=64) trong DE: 6.1908544279454246e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=64) trong DE: 2.4474010555678036e-06


[(array([ 2.95229512e-07, -2.94177487e-06]), 8.36262469450233e-06, 3713),
 (array([-1.04928532e-06,  9.27028074e-07]), 3.960234764566195e-06, 3649),
 (array([-2.05730683e-06,  1.13321654e-06]), 6.643453990040626e-06, 3777),
 (array([-1.79554269e-06, -4.61527010e-07]), 5.243739675808712e-06, 3841),
 (array([-5.03905921e-07, -5.65523862e-08]), 1.4342156160473962e-06, 3649),
 (array([-2.48668487e-06, -3.34910279e-07]), 7.097077709961042e-06, 3521),
 (array([3.06773669e-06, 1.10944641e-06]), 9.227148386248274e-06, 3841),
 (array([-3.81637149e-07, -1.31907405e-06]), 3.883969114770736e-06, 3457),
 (array([ 1.95496442e-06, -1.24166668e-06]), 6.5506371638157646e-06, 3585),
 (array([2.23512477e-07, 3.35313417e-06]), 9.505443163693172e-06, 3777)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=2, n=128) trong DE: 7.079780936880553e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=128) trong DE: 2.002365420690232e-06


[(array([ 6.73269335e-07, -2.90150924e-06]), 8.42498429465266e-06, 7041),
 (array([-9.08492787e-07,  1.15103060e-06]), 4.147567928658447e-06, 7553),
 (array([1.55740804e-06, 2.83968165e-06]), 9.160764744553518e-06, 6401),
 (array([2.74404707e-06, 5.89421395e-07]), 7.938578296506904e-06, 6657),
 (array([ 2.13195198e-06, -2.50712269e-06]), 9.30872906979019e-06, 7809),
 (array([ 3.74023174e-07, -1.54343955e-06]), 4.4919255484110465e-06, 7681),
 (array([-3.42802382e-06,  5.82485048e-07]), 9.835213380160468e-06, 6785),
 (array([1.90274405e-06, 1.03929142e-06]), 6.132375383050004e-06, 6913),
 (array([2.14266941e-06, 4.58703363e-07]), 6.197831283305533e-06, 7297),
 (array([ 1.08611736e-06, -1.46568334e-06]), 5.159839439716762e-06, 7937)]

In [ ]:
all_results = []
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=2, n=256) trong DE: 6.1832644729697964e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=256) trong DE: 2.3295205877005353e-06


[(array([-2.57481994e-06, -8.86587406e-08]), 7.287183312332246e-06, 14593),
 (array([-2.15670301e-07,  7.20626082e-07]), 2.1275782349228223e-06, 12545),
 (array([-2.63617526e-06,  4.15649407e-07]), 7.548532221068882e-06, 12801),
 (array([-1.03681132e-06,  4.46304013e-07]), 3.1927318357993784e-06, 14593),
 (array([-9.76165573e-07,  2.76506170e-06]), 8.294065427794095e-06, 14337),
 (array([-1.76469460e-06,  1.05559369e-06]), 5.816245377321394e-06, 14849),
 (array([-2.19909549e-06, -1.84914059e-06]), 8.126887980797193e-06, 14849),
 (array([3.10702656e-06, 1.30826946e-06]), 9.535577377040028e-06, 13313),
 (array([ 2.13660480e-06, -5.01649516e-07]), 6.207692873960724e-06, 13313),
 (array([-8.86278297e-07, -9.60291335e-07]), 3.6961500886611987e-06, 14337)]

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Ackley_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=16) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=16) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=10, n=16) trong DE: 9.225332440454536e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=16) trong DE: 6.392080650547643e-07


[(array([-2.50771596e-06, -3.98426661e-06,  1.53010694e-06, -1.53124110e-06,
         -9.55595807e-07, -1.28028034e-06,  2.39841868e-06, -7.92798845e-07,
         -3.50291953e-06, -3.12720386e-06]),
  9.619623459133919e-06,
  8417),
 (array([-4.73547542e-07,  2.67329258e-07, -1.17973480e-06, -3.13461281e-06,
          5.17775144e-06,  7.69990386e-07,  4.13784273e-07,  2.61480306e-06,
         -3.77318034e-06,  1.14089312e-06]),
  9.916957278921501e-06,
  8769),
 (array([-3.27704630e-06,  2.26237382e-06, -1.40075310e-06, -7.59254588e-07,
          1.53095823e-06,  5.75812110e-07,  5.61553508e-07, -9.04075078e-07,
          4.83835427e-06,  3.95421484e-06]),
  9.899782203337537e-06,
  8721),
 (array([-3.63058289e-06,  1.83035792e-06,  1.75875738e-06,  1.59690529e-06,
          1.77242965e-06, -1.10281919e-06,  2.60473516e-08,  3.66434525e-06,
          2.78369761e-06,  2.10671199e-06]),
  9.1346265773673e-06,
  8209),
 (array([-9.45436640e-07,  3.05849840e-06,  2.17045776e-06,  1.7430251

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=32) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=32) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=10, n=32) trong DE: 8.943980550757402e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=32) trong DE: 1.1059747812322734e-06


[(array([ 1.30981326e-07,  1.33317838e-06, -2.19262180e-06,  1.01198997e-06,
          3.73935059e-06,  3.71000059e-06, -1.84220686e-06, -1.55219745e-06,
         -1.47555681e-06,  1.15444067e-06]),
  8.455875264434809e-06,
  19073),
 (array([ 1.17108089e-06, -3.71873732e-06,  3.56813820e-07,  2.92150859e-06,
         -2.54755765e-07,  3.90371663e-06, -1.54218886e-06,  6.48639455e-07,
          3.32933908e-06, -2.82013616e-06]),
  9.879888971386919e-06,
  19297),
 (array([-2.12949788e-06, -4.31636609e-07,  6.69933854e-07, -9.61335452e-07,
         -1.52848108e-09, -8.29943218e-07, -5.42091229e-06,  1.49380557e-07,
         -4.90179444e-06, -1.58395797e-07]),
  9.818193469346426e-06,
  20065),
 (array([ 3.87705442e-06, -3.42051719e-06,  1.99549107e-06, -1.07816612e-06,
          1.71441651e-06, -5.51392245e-07, -1.00783383e-06,  1.46006407e-06,
         -8.54453333e-07, -6.22062744e-07]),
  7.938344709135237e-06,
  18945),
 (array([ 3.56013805e-06,  4.77203558e-07, -2.72584378e-06, -4.0

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Ackley_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=64) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=64) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=10, n=64) trong DE: 8.617646935693557e-06
Độ lệch chuẩn (std) của  Ackley (d=2, n=64) trong DE: 1.6474930330460783e-06


[(array([-3.47994762e-06, -4.98280184e-07, -1.12545840e-06, -2.49480469e-06,
         -1.89375528e-06,  1.44264367e-07, -1.92351531e-06,  4.25055674e-06,
          2.41688625e-06, -3.00754049e-06]),
  9.807250702653647e-06,
  39169),
 (array([-2.06302220e-06, -1.13726581e-07, -1.45318962e-06,  6.51517075e-07,
          2.78651707e-06,  2.16609755e-06,  2.01542152e-06, -1.92185424e-06,
          7.94976254e-07,  3.85185785e-06]),
  8.245091688952044e-06,
  40513),
 (array([-1.70683656e-07, -1.01939168e-06, -2.38667726e-06, -2.96040079e-06,
          3.24434817e-06,  2.94332319e-06,  2.25297684e-06, -1.21482622e-06,
         -3.34473449e-06, -2.15953368e-06]),
  9.560218999116188e-06,
  40513),
 (array([-2.20825078e-06,  7.55023945e-07, -1.99193318e-06,  2.72841012e-06,
         -2.23466384e-06,  3.36911341e-06,  6.47317082e-07, -3.89500678e-06,
         -3.26648810e-06, -3.91462232e-08]),
  9.754468547118478e-06,
  41089),
 (array([ 2.89405417e-06, -3.81519424e-06,  5.51293090e-07, -9.8

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Ackley_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=128) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=128) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=10, n=128) trong DE: 9.317907013173255e-06
Độ lệch chuẩn (std) của  Ackley (d=10, n=128) trong DE: 7.065370124892134e-07


[(array([-5.44011667e-06,  1.70263505e-07,  4.67127026e-07, -2.95821478e-06,
         -2.00767632e-07,  2.48514230e-06,  3.44394000e-07,  2.27362832e-06,
         -7.19260142e-07,  2.71914783e-06]),
  9.6342850075537e-06,
  80129),
 (array([ 5.51919827e-07,  3.36232345e-06, -2.76597296e-06,  1.85727437e-07,
         -2.62544999e-06,  3.67336797e-06, -2.70711452e-06,  9.25635865e-07,
         -7.88379120e-07, -2.97540643e-06]),
  9.578732090620434e-06,
  80257),
 (array([ 3.60257797e-06, -2.46021064e-06, -1.49427940e-06,  2.20079964e-08,
          2.77563758e-07, -1.99347293e-06,  3.48691829e-08, -1.50413136e-06,
          2.59130567e-06,  5.10486071e-06]),
  9.827111793203613e-06,
  81665),
 (array([-7.86159507e-07,  1.28511829e-07, -3.19814064e-07, -2.37373983e-06,
         -2.56335435e-06, -2.13959350e-06, -2.22184316e-06,  1.64238256e-06,
         -3.14203452e-06,  3.86973250e-06]),
  8.944658752785273e-06,
  78849),
 (array([-2.88925642e-06, -1.54415319e-06, -2.79273076e-07, -3.163

In [ ]:
all_results = []
d = 10
max_evals = 100000
f =   Ackley_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6

for i in range(10):
    np.random.rand(seed_number)
    results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
    all_results.append(results[-1])
    seed_number += 1

print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=256) trong DE: {np.mean([all_results[i][1] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=256) trong DE: {np.std([all_results[i][1] for i in range(len(all_results))])}")
all_results

Giá trị trung bình (mean) của  Ackley (d=10, n=256) trong DE: 0.0018257142511417079
Độ lệch chuẩn (std) của  Ackley (d=10, n=256) trong DE: 0.00031266184397776164


[(array([-3.90599319e-04, -1.29093401e-05,  6.91869980e-04,  9.88151653e-04,
         -5.10136570e-04, -1.71692507e-04, -4.73364468e-04,  3.55146067e-04,
         -2.08921407e-04, -5.15408233e-04]),
  0.002036319785166274,
  100097),
 (array([-1.62954935e-04, -4.62598649e-04,  3.11930418e-04,  2.14453111e-04,
         -5.32049768e-05, -4.09046498e-06,  2.92721071e-04, -3.64152621e-04,
         -9.96021439e-04, -1.23527161e-04]),
  0.0016147250137383828,
  100097),
 (array([ 3.74797138e-04,  1.36370881e-04,  5.35476192e-04, -3.49468542e-04,
          5.05650163e-05,  2.17238996e-04,  3.07618139e-04,  7.05464985e-04,
         -1.02392296e-03, -7.00602675e-04]),
  0.002111974968415087,
  100097),
 (array([ 2.69749419e-04,  4.51615840e-04,  3.95438276e-04,  7.56966496e-04,
          4.08126012e-04, -5.65446087e-04,  1.36211604e-03, -1.38333478e-04,
          8.43400319e-05, -1.83704817e-04]),
  0.0023531839624202355,
  100097),
 (array([-5.85389804e-04,  8.04680688e-05, -7.70972275e-05, -3

### **CEM**

In [ ]:
import tensorflow as tf
def CEMv2(fobj, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number):
    np.random.seed(seed_number)
    eps = 1e-4
    lower_bound, upper_bound = np.asarray(bounds).T
    sigma = sigma_init * np.eye(dimensions)

    diff = np.fabs(lower_bound - upper_bound)
    n_evals = 0
    num_evals = [0]
    mu = np.random.rand(dimensions) - (upper_bound + 1)
    generation_count = 0
    all_mu = []
    all_sigma = []
    all_offspring = []
    all_elite = []
    all_fitness = []
    while True:
    # for i in range(10000):
        if n_evals > max_evals:
            break
        all_mu.append(mu)
        all_sigma.append(sigma)

        x = np.random.multivariate_normal(mu, sigma, popsize)
        all_offspring.append(x)
        fitness = np.array([fobj(x[i]) for i in range(popsize)])
        n_evals += popsize
        best_fitness = min(fitness)
        all_fitness.append(best_fitness)
        # if np.sum(x) > 1e150 or np.sum(x) < -1e150:
        #     break
        if np.sum(x) > 1e150 or np.sum(x) < -1e150:
             break
        elite_idx = fitness.argsort()[:num_elite]
        all_elite.append(elite_idx)

        sigma = np.zeros_like(sigma)

        for i in range(num_elite):
            z = x[elite_idx[i]] - mu
            z = z.reshape(-1, 1)
            sigma += tf.matmul(z.T, z)

        sigma += np.eye(dimensions)*extra_std
        sigma *= (1/num_elite)
        mu = np.mean(x[elite_idx], axis=0)

        generation_count += 1
        num_evals.append(n_evals)

    all_mu.append(mu)
    best_results = mu.copy()
    best_fitness = fobj(mu)
    return all_mu, all_fitness, num_evals, generation_count

**Sphere**

In [ ]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.0321436  -0.00597042] , 2.5798382629683986e-05 , 960

[-0.00645473 -0.05299249] , 2.2454183970560288e-05 , 576

[-0.02375376  0.03738301] , 6.619181186942632e-05 , 1136

[ 0.00933059 -0.01085324] , 3.680318889197728e-05 , 592

[-0.03903084  0.02984113] , 5.396812453103912e-05 , 1120

[0.04171205 0.02291828] , 8.102475494590698e-05 , 288

[-0.03664636  0.03379384] , 7.543331639816228e-05 , 1024

[ 0.00527929 -0.01665478] , 2.87134269215555e-05 , 656

[-0.01945805 -0.00039376] , 5.893734942719276e-05 , 960

[ 0.01201922 -0.01167846] , 9.363033307271756e-05 , 1056

Giá trị trung bình (mean) của Sphere (d=2, n=16) trong CEMv2: 5.429548726582221e-05
Độ lệch chuẩn (std) của Sphere (d=2, n=16) trong CEMv2: 2.378664360012706e-05


In [ ]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")


[-0.01876765  0.01105757] , 8.53527765585344e-05 , 640

[ 0.01657663 -0.00490942] , 1.0275851008286996e-05 , 704

[9.47306828e-05 2.26357308e-02] , 4.189701172674215e-05 , 640

[ 0.00663253 -0.00122019] , 9.38905083809328e-05 , 896

[-0.00772806  0.0122609 ] , 2.30522052073879e-05 , 1472

[-0.03531526  0.02252861] , 5.1518376846544906e-05 , 608

[-0.01205526 -0.00545591] , 2.8803225948546123e-05 , 512

[-0.04835621  0.04783677] , 5.883234217996235e-05 , 256

[0.00814506 0.00234346] , 2.2820468105173874e-05 , 800

[0.05908312 0.02062982] , 2.8457935668360735e-05 , 224

Giá trị trung bình (mean) của Sphere (d=2, n=32) trong CEMv2: 4.449007016304722e-05
Độ lệch chuẩn (std) của Sphere (d=2, n=32) trong CEMv2: 2.6409957972365036e-05


In [ ]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")


[ 0.00717147 -0.00233088] , 7.793072343689408e-05 , 1216

[ 0.03178297 -0.01926316] , 3.07722527392638e-05 , 1024

[-0.00150855 -0.00322259] , 3.363947810613169e-05 , 1088

[ 0.02548322 -0.00919982] , 1.5621608351842697e-06 , 640

[-0.0080788   0.01575485] , 1.7625992510809682e-05 , 2176

[ 0.00955545 -0.0040204 ] , 1.614464441402927e-05 , 320

[ 0.03425744 -0.04600438] , 7.609088403573047e-05 , 576

[-0.02052186  0.02706156] , 3.556923534228687e-05 , 768

[ 0.01629206 -0.03199474] , 1.3343215474763267e-05 , 640

[ 0.05773016 -0.04978664] , 7.964542417518276e-05 , 960

Giá trị trung bình (mean) của Sphere (d=2, n=64) trong CEMv2: 3.823240110702762e-05
Độ lệch chuẩn (std) của Sphere (d=2, n=64) trong CEMv2: 2.772142198077061e-05


In [ ]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.01500624 -0.02841917] , 3.931716735223535e-06 , 2688

[ 0.02312585 -0.03356441] , 9.359282323585253e-05 , 1536

[-0.04215873  0.04211769] , 1.4158372914487866e-05 , 2176

[-0.0242589  0.0122632] , 5.5966067632652475e-05 , 1408

[-0.04927922  0.0504712 ] , 3.635641741936223e-05 , 1024

[-0.00045869 -0.00174397] , 6.185369152184831e-05 , 1792

[-0.05970537  0.02905831] , 3.6524225184872478e-06 , 1152

[ 0.01368438 -0.0336175 ] , 9.467089954575791e-05 , 512

[ 0.02192271 -0.02600138] , 8.753792999716619e-05 , 1408

[-0.0030717   0.00246877] , 5.3878090517966466e-06 , 2560

Giá trị trung bình (mean) của Sphere (d=2, n=128) trong CEMv2: 4.57108150572635e-05
Độ lệch chuẩn (std) của Sphere (d=2, n=128) trong CEMv2: 3.609495034443633e-05


In [ ]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=2, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=2, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.03709662 -0.04620946] , 9.448478054856378e-05 , 2560

[ 0.02134897 -0.02162562] , 6.707093950277304e-05 , 1792

[-0.02915403  0.02223319] , 4.616333561301784e-05 , 2560

[-0.05559067  0.04370434] , 3.053442419444982e-05 , 1792

[ 0.06266521 -0.06703539] , 1.133857005134247e-05 , 4096

[ 0.02407541 -0.029609  ] , 8.471526719874213e-05 , 1536

[-0.03640061  0.02790041] , 9.363093734130777e-05 , 3328

[-0.0200418   0.01702671] , 5.243124317904683e-06 , 2816

[-0.01323372  0.01892489] , 8.222872145250674e-06 , 2304

[ 0.01495183 -0.02396247] , 2.2227974598013514e-05 , 5120

Giá trị trung bình (mean) của Sphere (d=2, n=256) trong CEMv2: 4.6363222551136573e-05
Độ lệch chuẩn (std) của Sphere (d=2, n=256) trong CEMv2: 3.413939977807883e-05


In [ ]:
d = 10
max_evals = 100000
f =   Sphere_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")


[-9.36283963e+146 -9.36284103e+146 -9.36284103e+146 -9.36284103e+146
 -9.36284103e+146 -9.36284103e+146 -9.36284103e+146 -9.36284103e+146
 -9.36284103e+146 -9.36284103e+146] , 4.5378984669571283e+294 , 12208

[1.42388073e+147 1.42388070e+147 1.42388070e+147 1.42388070e+147
 1.42388070e+147 1.42388070e+147 1.42388070e+147 1.42388070e+147
 1.42388070e+147 1.42388070e+147] , 1.6180171328706247e+295 , 11808

[-7.99527951e+146 -7.99527788e+146 -7.99527788e+146 -7.99527788e+146
 -7.99527788e+146 -7.99527788e+146 -7.99527788e+146 -7.99527788e+146
 -7.99527788e+146 -7.99527788e+146] , 2.1599328843387924e+296 , 12144

[3.49361955e+147 3.49361955e+147 3.49361955e+147 3.49361955e+147
 3.49361955e+147 3.49361955e+147 3.49361955e+147 3.49361955e+147
 3.49361955e+147 3.49361955e+147] , 4.635463842957387e+294 , 12496

[-3.29037381e+147 -3.29037379e+147 -3.29037379e+147 -3.29037379e+147
 -3.29037379e+147 -3.29037379e+147 -3.29037379e+147 -3.29037379e+147
 -3.29037379e+147 -3.29037379e+147] , 1.2329571

In [ ]:
d = 10
max_evals = 100000
f =   Sphere_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.06731192 -0.02781933 -0.09176115  0.01413891  0.0086449  -0.05772501
  0.03197451 -0.02630179  0.03722141  0.0188869 ] , 0.020960795203764107 , 100032

[ 0.02475228 -0.03381091  0.00984662  0.01254246  0.00355788 -0.01079011
  0.05105845 -0.0730542   0.07081823 -0.02842236] , 0.0170144406730862 , 100032

[ 0.02824045  0.0327735  -0.01827913  0.00903631  0.01123297  0.06681426
 -0.02348541  0.0141041   0.0297093   0.04051378] , 0.00958169351085341 , 100032

[ 0.04824264 -0.04294249  0.00880298 -0.0384473  -0.04621838  0.01970524
  0.03573521 -0.01312174 -0.09797881 -0.00331055] , 0.019697940250639603 , 100032

[-0.03964752 -0.03535898 -0.02429115  0.05355687  0.05116252  0.05400473
  0.00660948 -0.0232943   0.0322702  -0.00745824] , 0.013531912659923137 , 100032

[ 0.06628249  0.0412033   0.04732341 -0.00156526 -0.02745841  0.01516404
 -0.00826628  0.04951764  0.02939163 -0.04492927] , 0.018913937437278282 , 100032

[-0.03058939 -0.05821908  0.03992731  0.05488778 -0.05744106 -0.069

In [ ]:
d = 10
max_evals = 100000
f =   Sphere_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.01213757 -0.00829879  0.01211329 -0.01517744 -0.007395   -0.00332442
 -0.00249852 -0.00382416  0.00016167  0.00767153] , 0.0051624241889434526 , 100032

[ 0.00115537 -0.01785945  0.00816    -0.0027956   0.02573715 -0.02769345
 -0.00887817  0.00638698  0.0246519  -0.00398366] , 0.006710096237454548 , 100032

[ 0.02092857  0.00198225  0.00125727  0.0144025   0.00419576  0.00268717
  0.00386975  0.01153764 -0.01565765  0.00291143] , 0.004933548132984919 , 100032

[-0.01565808 -0.01088668  0.00187571 -0.00296093  0.00418129 -0.0116437
  0.01122551 -0.0137725  -0.00464004 -0.01315926] , 0.0053010700534968196 , 100032

[-0.00256201  0.0169345  -0.01197597  0.01968416 -0.0106907   0.00089055
 -0.02627802  0.03044629 -0.03015167  0.00944555] , 0.009706188263876694 , 100032

[ 5.91780323e-03 -2.24005763e-02  6.26282081e-03 -2.33834060e-03
 -8.92256765e-04  1.97665629e-02 -2.19616603e-02  1.11054937e-02
  3.56495420e-05  1.48986909e-02] , 0.004687399725554723 , 100032

[-0.00654775  0.000639

In [ ]:
d = 10
max_evals = 100000
f =   Sphere_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.00184531 -0.01543686  0.00781962 -0.00516657  0.00460129  0.00740594
  0.00381717  0.00953815  0.00647833 -0.00055284] , 0.004622144699755238 , 100096

[ 0.00077912  0.00144562 -0.01307766 -0.00998957  0.0164112  -0.00576964
  0.00701673 -0.01461368  0.00956265  0.00387526] , 0.002747793503463917 , 100096

[-0.00721331  0.00306878 -0.00393533 -0.00549076  0.01823803  0.01555337
  0.00660508  0.00782622  0.01949662  0.00495118] , 0.0036560951288183118 , 100096

[-0.01712412 -0.00016062 -0.01485248 -0.0061062  -0.02090156  0.00121623
  0.0022906  -0.00663368  0.0034392   0.0015577 ] , 0.0022747783611429175 , 100096

[ 0.00831557  0.01238888  0.00614597 -0.00066285 -0.00486271  0.00155184
  0.00955459 -0.0084763  -0.00628539 -0.01859911] , 0.005470371463163832 , 100096

[-0.01979875 -0.02085927  0.01010685  0.0068011  -0.00435628  0.00944447
  0.00375406  0.00297127 -0.0036094   0.0059257 ] , 0.004984553869280329 , 100096

[-0.01222121 -0.00091992 -0.00507654 -0.00531792 -0.01063991 -

In [ ]:
d = 10
max_evals = 100000
f =   Sphere_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Sphere (d=10, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Sphere (d=10, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.00323017  0.00025929 -0.00371124 -0.00715589 -0.0150003   0.0037968
 -0.00285635  0.00893146 -0.01461366  0.00054984] , 0.0021672483748896632 , 100096

[-0.0104136  -0.0152059  -0.0113424  -0.00511312  0.00649038 -0.00402658
  0.00427008  0.00428014 -0.00517049  0.00250136] , 0.0036818391806792837 , 100096

[ 0.01274179  0.00840736 -0.00227846 -0.00783039  0.0025442   0.00685582
  0.00395306  0.00744029 -0.0049691   0.00959659] , 0.002125891879388527 , 100096

[-0.01086877  0.01006742  0.0025547   0.00908914 -0.00583346  0.0137469
  0.00692934  0.00076933  0.00389315 -0.00562304] , 0.0024040602695325213 , 100096

[-0.00613965 -0.00675085 -0.00104105 -0.00248508  0.00828912 -0.008984
  0.0051201  -0.01154256  0.00112285  0.00829484] , 0.0027811850775467015 , 100096

[-1.31716646e-02  7.82686174e-04  2.30102502e-03 -3.83838993e-04
  2.98612112e-03 -2.59762602e-03  1.13255081e-03  1.13997744e-02
 -7.10379664e-05 -1.74468046e-03] , 0.0042047083106026475 , 100096

[-0.00864698  0.009417

**Griewank**

In [ ]:
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-3.26569668 -4.59257933] , 0.25685469418394846 , 20016

[-17.07135883 -17.55367578] , 0.5702414914604809 , 20016

[-0.43728719  1.48307232] , 0.08384117706784222 , 20016

[-4.67006634 -3.81558314] , 0.32112866988227085 , 20016

[2.88629208 4.22522085] , 0.3612957356522549 , 20016

[-3.08184574 -2.82941248] , 0.2612803373158955 , 20016

[-2.22463549 -2.27760437] , 0.27038036590698467 , 20016

[3.77286918 2.31470027] , 0.05335758599314988 , 20016

[-5.52948252 -5.94918293] , 0.6341295889365657 , 20016

[-4.1118275  -4.57298073] , 0.17265425644981636 , 20016

Giá trị trung bình (mean) của Griewank (d=2, n=16) trong CEMv2: 0.29851639028492094
Độ lệch chuẩn (std) của Griewank (d=2, n=16) trong CEMv2: 0.17800750848405492


In [ ]:
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-3.14395176 -4.43138161] , 0.007423686337789115 , 20032

[-6.27452546 -8.88838718] , 0.02965801021886627 , 20032

[-0.03543786  0.08646993] , 7.978260574059703e-05 , 1792

[-0.01823446  0.00464704] , 9.836238664595598e-05 , 2464

[-0.03699772  0.05704829] , 4.0952731956922506e-05 , 1696

[-3.13839524 -4.43545851] , 0.007420183550144621 , 20032

[ 0.01594594 -0.04225873] , 1.6022835898632515e-05 , 1824

[-3.14587663 -4.42762998] , 0.007443586344244579 , 20032

[3.14211614 4.43265065] , 0.007444375582580931 , 20032

[ 0.01759042 -0.05386862] , 2.4834658152284028e-05 , 2880

Giá trị trung bình (mean) của Griewank (d=2, n=32) trong CEMv2: 0.005964979725201991
Độ lệch chuẩn (std) của Griewank (d=2, n=32) trong CEMv2: 0.00863019280318342


In [ ]:
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.02674701 -0.02880546] , 8.48508870898712e-06 , 1792

[-3.13960176 -4.43930678] , 0.00744538091261937 , 20032

[3.14200019 4.44157986] , 0.007407870139459538 , 20032

[3.13833694 4.44047335] , 0.00739643980210547 , 20032

[ 0.0155116  -0.01796674] , 5.319158682648428e-05 , 1024

[ 0.03076493 -0.03545615] , 5.605538710007352e-05 , 576

[ 0.02014183 -0.05411915] , 7.237970295015383e-05 , 1216

[-0.00402411  0.0362207 ] , 5.143978605959276e-05 , 832

[ 0.00534486 -0.01866136] , 1.4169720592427382e-06 , 1216

[-3.13638808 -4.43512849] , 0.007397901013578512 , 20032

Giá trị trung bình (mean) của Griewank (d=2, n=64) trong CEMv2: 0.0029890560391467426
Độ lệch chuẩn (std) của Griewank (d=2, n=64) trong CEMv2: 0.003611313728648264


In [ ]:
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.02612794 -0.06770517] , 1.4556255857889333e-05 , 1280

[-6.27893558 -8.87276167] , 0.029614760356238556 , 20096

[3.14389481 4.4365788 ] , 0.007404333450213074 , 20096

[-0.01785215  0.03409212] , 3.639803023447463e-05 , 1664

[-0.017149   0.0474049] , 1.929226077024282e-05 , 1792

[3.13905879 4.43691514] , 0.007398060786459881 , 20096

[-3.14065914 -4.43813297] , 0.007401331016126389 , 20096

[3.13974876 4.44378675] , 0.007404503322151812 , 20096

[ 0.03414104 -0.06740649] , 6.93737203849576e-05 , 1152

[-3.13989325 -4.44321659] , 0.007396713528221022 , 20096

Giá trị trung bình (mean) của Griewank (d=2, n=128) trong CEMv2: 0.00667593227266583
Độ lệch chuẩn (std) của Griewank (d=2, n=128) trong CEMv2: 0.008397816585217386


In [ ]:
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=2, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=2, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-3.13932666 -4.43727597] , 0.007397728700977213 , 20224

[-3.13882776 -4.43942867] , 0.007404451594900219 , 20224

[-0.01422898  0.06019703] , 9.474731498371192e-06 , 1536

[-0.02925893  0.00602251] , 7.315303785782046e-05 , 1024

[-3.1411756  -4.44099712] , 0.007397748008317828 , 20224

[-0.03001961  0.04739833] , 8.446827940056245e-05 , 2816

[-3.14152489 -4.43657591] , 0.007396779474016513 , 20224

[3.13978506 4.44149495] , 0.00739794949910022 , 20224

[-3.14095641 -4.43711157] , 0.0074006280696912174 , 20224

[-3.13803354 -4.43809703] , 0.007399090199267322 , 20224

Giá trị trung bình (mean) của Griewank (d=2, n=256) trong CEMv2: 0.005196147159502729
Độ lệch chuẩn (std) của Griewank (d=2, n=256) trong CEMv2: 0.0033652626483819093


In [ ]:
d = 10
max_evals = 100000
f =   Griewank_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-9.28982564e+146 -9.28982702e+146 -9.28982702e+146 -9.28982702e+146
 -9.28982702e+146 -9.28982702e+146 -9.28982702e+146 -9.28982702e+146
 -9.28982702e+146 -9.28982702e+146] , 1.1168497194571318e+291 , 12208

[1.43593811e+147 1.43593809e+147 1.43593809e+147 1.43593809e+147
 1.43593809e+147 1.43593809e+147 1.43593809e+147 1.43593809e+147
 1.43593809e+147 1.43593809e+147] , 4.113839542288969e+291 , 11808

[-8.65988041e+146 -8.65987866e+146 -8.65987866e+146 -8.65987866e+146
 -8.65987866e+146 -8.65987866e+146 -8.65987866e+146 -8.65987866e+146
 -8.65987866e+146 -8.65987866e+146] , 6.334856091224038e+292 , 12144

[3.37039902e+147 3.37039901e+147 3.37039901e+147 3.37039901e+147
 3.37039901e+147 3.37039901e+147 3.37039901e+147 3.37039901e+147
 3.37039901e+147 3.37039901e+147] , 1.0785607882661514e+291 , 12496

[-3.53659558e+147 -3.53659561e+147 -3.53659561e+147 -3.53659561e+147
 -3.53659561e+147 -3.53659561e+147 -3.53659561e+147 -3.53659561e+147
 -3.53659561e+147 -3.53659561e+147] , 3.56096998

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [ ]:
d = 10
max_evals = 100000
f =   Griewank_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.02455587 -0.01527331 -0.0177226  -0.0268985   0.00154394  0.04623529
 -0.00486584  0.02072977 -0.07447805 -0.0038631 ] , 0.0010574786535453295 , 100032

[-0.03823851  0.01898175  0.02268106  0.07161167  0.03994764 -0.03033568
  0.01333635  0.00709238  0.13799667  0.07640444] , 0.003126244342867568 , 100032

[ 0.04959668 -0.01327533  0.02122182  0.01225195  0.03310419 -0.00109054
  0.02575335  0.0286808   0.03943214 -0.02219551] , 0.0015710204992133159 , 100032

[-0.03374346 -0.00388714 -0.00787565 -0.07684514  0.00749265  0.1475667
  0.04475218  0.08506725 -0.0706366  -0.0070271 ] , 0.0025872823626222896 , 100032

[ 0.00675563  0.03498644 -0.02776593 -0.05066997 -0.01948141  0.08154678
  0.02677813  0.05564651  0.05910001  0.03798633] , 0.0020446976305736886 , 100032

[ 0.01628962  0.02238154  0.08072476 -0.01651178 -0.06185673  0.04971967
 -0.0454544  -0.018785   -0.01209741  0.09995596] , 0.0019834448944374694 , 100032

[-0.03735228 -0.05352068 -0.11216756  0.0050638  -0.02474549

In [ ]:
d = 10
max_evals = 100000
f =   Griewank_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.00104705 -0.01099069  0.00221155 -0.00035591  0.04276009 -0.03757736
 -0.04075769  0.01753793  0.01415639 -0.01375549] , 0.0008137297500248053 , 100032

[ 0.00064124 -0.00284984  0.00627033 -0.01135442 -0.01263525 -0.00276931
 -0.00724518  0.0360308   0.01301448  0.00867366] , 0.0005909659079760265 , 100032

[ 0.0066683   0.00395276 -0.00145139  0.008621   -0.00648586 -0.01390707
 -0.01776995  0.01862128  0.00491009  0.01199034] , 0.0006982474410650585 , 100032

[ 0.00189821 -0.00764788  0.00456653  0.01958251  0.01064112 -0.0437485
 -0.0270687   0.04396065  0.00954277  0.01917807] , 6.062112682214682e-05 , 10048

[ 0.00130696  0.02802102 -0.0079084  -0.03275302 -0.0072466  -0.00171526
 -0.00936823  0.00401099  0.03201315  0.02160973] , 8.370208176722294e-05 , 32640

[ 0.00675009  0.00768804  0.02092683  0.01962728  0.00936537 -0.0110654
 -0.03791817 -0.00535612  0.00762518  0.02537799] , 0.0008258304608801392 , 100032

[-0.00957932 -0.02399934 -0.00495342 -0.03160965  0.00276488 -

In [ ]:
d = 10
max_evals = 100000
f =   Griewank_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.00450726  0.00102115  0.00517202  0.01175786 -0.00175043 -0.01708325
  0.00959548  0.01504083  0.00361843  0.02324461] , 0.0004841030378694988 , 100096

[ 0.00446126  0.01110932 -0.00516668 -0.01246483 -0.00571844  0.00701441
  0.00664816 -0.01016474  0.00664263  0.00457648] , 0.0007079498478116397 , 100096

[ 0.00138701 -0.00591233  0.00801033 -0.00834163 -0.00909286  0.03133037
 -0.00858912  0.00443863 -0.01689137  0.03193537] , 0.000484266882766371 , 100096

[-0.00713722  0.001435   -0.01487335  0.00309203 -0.03226133 -0.00194276
 -0.01250675 -0.00502367  0.01800986 -0.01210669] , 0.00018914371146250275 , 100096

[ 0.00149038 -0.00982644  0.00136512  0.01078511 -0.00127956  0.00282135
 -0.01157079  0.00696372  0.02082369 -0.00441052] , 9.160832515975326e-05 , 22656

[-0.00302944  0.00074769 -0.00750881 -0.0082051  -0.02967557  0.00609885
 -0.00205192  0.03029238  0.01348906  0.02133609] , 0.0005067244989189756 , 100096

[-0.01259501  0.00515696  0.00470031  0.00058007 -0.0160878

In [ ]:
d = 10
max_evals = 100000
f =   Griewank_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Griewank (d=10, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Griewank (d=10, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.00646808 -0.00410743  0.0016553   0.00453621  0.00479758 -0.00124245
  0.00687261  0.00977148 -0.00367899  0.01113176] , 0.0004738435265683183 , 100096

[-0.00060061 -0.00019051  0.00234584  0.0071862  -0.00136434 -0.00051811
  0.01714652 -0.01000282 -0.0018567  -0.00641537] , 9.851113093672925e-05 , 73216

[-0.00459253  0.00305871  0.00370231 -0.00139147 -0.01525394 -0.00048462
  0.00580898  0.00277128  0.00995792  0.00541538] , 7.189829407250325e-05 , 63488

[-0.01009104 -0.0147133  -0.00473373 -0.00986643 -0.00940978 -0.00066576
 -0.00112223  0.00112316 -0.020883    0.00742542] , 8.466029043519008e-05 , 24576

[-5.26542691e-04  1.59691026e-05 -7.50521155e-03  5.48022641e-04
 -2.87180347e-02 -1.00452507e-02  1.35672843e-02  1.46449596e-02
 -2.85340148e-03 -1.11750058e-02] , 0.0003206799244495073 , 100096

[-0.00471896  0.0076532   0.00588369 -0.00876805  0.00416595 -0.00552321
  0.01180024 -0.00869096  0.00835967 -0.00042605] , 8.954912123038117e-05 , 87040

[ 0.00061501 -0.00608

**Rosenbrock**

In [ ]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=2, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=2, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.65281399 0.42866647] , 0.12218849574392773 , 20016

[0.48968243 0.27911134] , 0.08231460681401145 , 20016

[0.6346643  0.41741179] , 0.12923080770506995 , 20016

[0.46930839 0.2812362 ] , 0.061502796413290796 , 20016

[0.57931404 0.36635731] , 0.12021939617546085 , 20016

[0.56866926 0.32537265] , 0.17055235691585058 , 20016

[0.57350292 0.34968947] , 0.09334080190900647 , 20016

[0.56991503 0.3475483 ] , 0.13821468719358623 , 20016

[0.50445238 0.26786849] , 0.1467109851716068 , 20016

[0.55572833 0.32568844] , 0.08402341643278255 , 20016

Giá trị trung bình (mean) của Rosenbrock (d=2, n=16) trong CEMv2: 0.11482983504745933
Độ lệch chuẩn (std) của Rosenbrock (d=2, n=16) trong CEMv2: 0.032047887365695395


In [ ]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=2, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=2, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.9438269  0.88690229] , 2.6229781732193046e-05 , 7840

[0.99980553 0.99660422] , 9.832267914372078e-05 , 6944

[0.92330992 0.84826921] , 0.007864054738527252 , 20032

[0.77665261 0.60374422] , 0.047440283092539254 , 20032

[0.76253873 0.58861533] , 0.04059877692464825 , 20032

[0.9274607  0.86102399] , 4.8184727102486365e-05 , 8192

[0.97328894 0.94235022] , 5.8515376826635045e-05 , 7296

[0.9640338  0.93180871] , 7.703878525732766e-05 , 10848

[0.95837125 0.91083783] , 9.859350151158338e-05 , 12704

[0.96655456 0.94296162] , 7.956117306464358e-05 , 11584

Giá trị trung bình (mean) của Rosenbrock (d=2, n=32) trong CEMv2: 0.009638956078035334
Độ lệch chuẩn (std) của Rosenbrock (d=2, n=32) trong CEMv2: 0.017411570430798006


In [ ]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=2, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=2, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.99198974 0.98235799] , 9.677640769879012e-05 , 7616

[0.9648616  0.93137321] , 5.058261473208297e-05 , 6080

[0.95254623 0.91262795] , 9.048791368741435e-05 , 3776

[0.97749354 0.95817481] , 8.835948285693677e-05 , 14848

[0.96258566 0.92439989] , 8.68623604805352e-05 , 6208

[0.97731097 0.95008029] , 3.522362801289419e-05 , 3776

[0.94215892 0.88979962] , 1.0597789968181429e-05 , 5248

[0.97488695 0.95168158] , 8.520397163162575e-06 , 4288

[0.98421826 0.96993026] , 2.512955147506485e-05 , 10240

[0.9433904 0.88952  ] , 3.298603002379457e-05 , 8448

Giá trị trung bình (mean) của Rosenbrock (d=2, n=64) trong CEMv2: 5.25526176098857e-05
Độ lệch chuẩn (std) của Rosenbrock (d=2, n=64) trong CEMv2: 3.315450957364904e-05


In [ ]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=2, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=2, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[1.01419975 1.03016627] , 4.97150749292411e-06 , 10752

[0.97411663 0.94913026] , 5.5258894861341425e-05 , 6528

[0.94005545 0.88421595] , 2.8695453037130497e-05 , 6400

[0.99517656 0.99498762] , 2.4071241050446592e-05 , 9472

[0.99515866 0.99259806] , 6.351090573451615e-05 , 8960

[0.99555591 0.99239112] , 6.825239128503912e-05 , 7040

[0.97110035 0.94126661] , 7.196740158199754e-05 , 5632

[0.97487945 0.94902593] , 9.232936206801987e-05 , 8320

[0.99024007 0.97961016] , 5.3040431406901434e-05 , 9856

[0.98805188 0.977759  ] , 3.217766856478964e-05 , 9984

Giá trị trung bình (mean) của Rosenbrock (d=2, n=128) trong CEMv2: 4.942752570831064e-05
Độ lệch chuẩn (std) của Rosenbrock (d=2, n=128) trong CEMv2: 2.5089925894866887e-05


In [ ]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=2, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=2, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.98835322 0.97377074] , 1.613925395078997e-05 , 14336

[0.99508365 0.99157318] , 7.734046986814812e-05 , 11008

[0.98317393 0.96715658] , 1.2897763932926038e-05 , 13824

[0.93802587 0.88227139] , 4.953600468047569e-05 , 16128

[0.95013217 0.89918042] , 1.8638371050037812e-05 , 9472

[0.99781053 0.99633295] , 8.645019615354382e-05 , 14080

[0.98519859 0.96992466] , 6.371082373417427e-05 , 11264

[0.97743198 0.95697748] , 8.874295131749109e-05 , 14080

[0.98592091 0.97131633] , 4.095073027141428e-05 , 9472

[0.95122932 0.90636779] , 6.551736477320253e-05 , 12032

Giá trị trung bình (mean) của Rosenbrock (d=2, n=256) trong CEMv2: 5.199239297322036e-05
Độ lệch chuẩn (std) của Rosenbrock (d=2, n=256) trong CEMv2: 2.748635253588149e-05


In [ ]:
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=10, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=10, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

<ipython-input-2-3c26f9ff247d>:18: RuntimeWarning: overflow encountered in scalar add
  f += (100*((x[i + 1] - x[i]**2)**2) + (1 -x[i])**2)
<ipython-input-2-3c26f9ff247d>:18: RuntimeWarning: overflow encountered in scalar multiply
  f += (100*((x[i + 1] - x[i]**2)**2) + (1 -x[i])**2)
<ipython-input-2-3c26f9ff247d>:18: RuntimeWarning: overflow encountered in scalar power
  f += (100*((x[i + 1] - x[i]**2)**2) + (1 -x[i])**2)


[4.17808699e+147 4.17808720e+147 4.17808720e+147 4.17808720e+147
 4.17808720e+147 4.17808720e+147 4.17808720e+147 4.17808720e+147
 4.17808720e+147 4.17808720e+147] , inf , 8576

[5.12019374e+147 5.12019358e+147 5.12019358e+147 5.12019358e+147
 5.12019358e+147 5.12019358e+147 5.12019358e+147 5.12019358e+147
 5.12019358e+147 5.12019358e+147] , inf , 8432

[1.15400398e+148 1.15400385e+148 1.15400385e+148 1.15400385e+148
 1.15400385e+148 1.15400385e+148 1.15400385e+148 1.15400385e+148
 1.15400385e+148 1.15400385e+148] , inf , 8656

[-3.90274697e+147 -3.90274710e+147 -3.90274710e+147 -3.90274710e+147
 -3.90274710e+147 -3.90274710e+147 -3.90274710e+147 -3.90274710e+147
 -3.90274710e+147 -3.90274710e+147] , inf , 8912

[4.37846141e+147 4.37846139e+147 4.37846139e+147 4.37846139e+147
 4.37846139e+147 4.37846139e+147 4.37846139e+147 4.37846139e+147
 4.37846139e+147 4.37846139e+147] , inf , 8384

[-3.29865964e+147 -3.29865929e+147 -3.29865929e+147 -3.29865929e+147
 -3.29865929e+147 -3.29865929e+

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [ ]:
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=10, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=10, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.99575261 0.95208098 0.97750713 0.93089727 0.85263484 0.78951486
 0.57404695 0.32145644 0.11122744 0.00989455] , 4.15600342428928 , 100032

[ 0.97131768  0.93179951  0.88407509  0.73976458  0.6072393   0.40909248
  0.15449559  0.04207505  0.05735488 -0.01178491] , 4.429118239488752 , 100032

[ 0.98928536  0.91849268  0.82517613  0.66031125  0.43727443  0.1554907
  0.02705503  0.11957325  0.06906583 -0.01506068] , 7.154681620337596 , 100032

[ 0.96231308  0.95901672  0.97176308  0.93924059  0.86794901  0.78991492
  0.65340781  0.39587517  0.10804307 -0.00599772] , 2.959644948532856 , 100032

[ 0.92051171  0.83112005  0.62901907  0.47340305  0.23620131  0.04467617
  0.06783496  0.02684381 -0.01766809  0.0046533 ] , 6.361055853244414 , 100032

[0.97663103 0.94065723 0.93122376 0.82265429 0.75767463 0.64298601
 0.45402333 0.2843464  0.05533039 0.05091867] , 3.624009030262033 , 100032

[ 0.96412511  0.94746782  0.91394682  0.87538062  0.79456721  0.71112223
  0.51496099  0.37992046  0.100

In [ ]:
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=10, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=10, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.99181258 0.99658924 0.97510081 0.95437296 0.93353772 0.85809072
 0.74050331 0.53214974 0.26123733 0.09309368] , 2.150367644854002 , 100032

[1.00147953 0.99612789 0.9939414  0.96220884 0.9269827  0.87498965
 0.75375707 0.58689418 0.36547744 0.14116116] , 1.7228258823227292 , 100032

[1.01858231 1.00507692 0.96340095 0.93234428 0.875002   0.7600662
 0.57479241 0.2999099  0.12935656 0.03249426] , 2.8015075812271903 , 100032

[0.96637677 0.97013706 0.971134   0.95392009 0.89547672 0.79485356
 0.62410885 0.41545818 0.1898538  0.01866822] , 1.9893309805763104 , 100032

[1.00438784 0.99004496 0.97928128 0.97347212 0.93510028 0.88143671
 0.79194802 0.64322651 0.44872832 0.18769378] , 1.8419095332001736 , 100032

[0.98543023 0.95564841 0.96342836 0.96902421 0.91613823 0.82121688
 0.68432878 0.46177683 0.23019905 0.06654325] , 2.479536712123234 , 100032

[0.97468138 0.97667942 0.95656798 0.91086642 0.84214005 0.72150807
 0.51664797 0.29086171 0.09973324 0.02464953] , 2.3950052562287363 , 100

In [ ]:
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=10, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=10, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.99120403 0.97770884 0.97004129 0.94784415 0.94351234 0.89598146
 0.79998416 0.65701223 0.45957154 0.20550032] , 2.297357523300084 , 100096

[0.97498026 0.98010557 0.96960107 0.95676635 0.93035289 0.85146208
 0.7279444  0.51480051 0.26928039 0.05086901] , 2.282894889609654 , 100096

[0.99314858 0.99809728 0.98378725 0.95969696 0.95115598 0.90715135
 0.82934442 0.70173722 0.49022723 0.2283465 ] , 1.170881475307806 , 100096

[1.00143184 0.98952453 0.98086551 0.9435626  0.89994111 0.82422192
 0.67080222 0.45975958 0.21087185 0.07164942] , 1.5068772843036413 , 100096

[0.98184471 0.99550437 0.97140035 0.96986045 0.93226901 0.85689882
 0.79149213 0.6336555  0.42217173 0.18282906] , 1.8766483629431723 , 100096

[0.98810636 1.0004319  0.99727983 0.9745276  0.96597519 0.92945653
 0.82182431 0.71598305 0.50878663 0.27319006] , 1.6589328565850927 , 100096

[0.98267608 0.98510669 0.96423366 0.96370291 0.94636207 0.91248317
 0.82281523 0.69018375 0.47766089 0.24956441] , 2.223495921745773 , 1000

In [ ]:
d = 10
max_evals = 100000
f =   Rosenbrock_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Rosenbrock (d=10, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Rosenbrock (d=10, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.99594552 0.99128582 0.97704055 0.95739757 0.90461898 0.85098629
 0.74076138 0.56571507 0.33237673 0.10858656] , 1.618246160596217 , 100096

[0.99540595 0.99613313 0.99369379 0.97685157 0.93794639 0.89589725
 0.7839041  0.59173242 0.35975084 0.11623252] , 1.4725317656241834 , 100096

[1.01308828 0.99748081 0.98094021 0.9598293  0.93101643 0.87349572
 0.75415933 0.58421842 0.34893666 0.10863701] , 1.319977114020547 , 100096

[0.99046805 0.99695627 0.98376148 0.94973973 0.90288468 0.82264761
 0.67889242 0.45778391 0.20624594 0.02865602] , 1.7868451509857524 , 100096

[0.97766265 0.97232872 0.99625402 0.97880114 0.9548304  0.93519835
 0.8683881  0.76419749 0.59974214 0.34369644] , 0.9648627336930443 , 100096

[0.98958536 0.98935669 0.98510472 0.97808642 0.95390738 0.90500943
 0.82972668 0.69402898 0.48532471 0.22406415] , 1.5472007275920643 , 100096

[0.99174366 0.98661353 0.97265924 0.94071537 0.8970707  0.82364394
 0.67837213 0.46382949 0.21254383 0.05518544] , 2.1072168134963754 , 10

**Michalewicz**

In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=2, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=2, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

<ipython-input-44-f3656a08da0b>:25: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  x = np.random.multivariate_normal(mu, sigma, popsize)


LinAlgError: SVD did not converge

In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=2, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=2, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[3.42317561e+80 3.42317561e+80] , -0.9347086983744356 , 20032

[-1.519108e+79 -1.519108e+79] , -1.1116702387409443 , 20032

[3.00809174e+78 3.00809174e+78] , -0.8264427055964412 , 20032

[-3.16643239e+81 -3.16643239e+81] , -0.671874366131436 , 20032

[-2.83214282e+84 -2.83214282e+84] , -0.9993616347785971 , 20032

[-4.96834658 -4.7096126 ] , -1.9623973176835072 , 20032

[-4.54969633e+77 -4.54969633e+77] , -0.8718863978649082 , 20032

[-4.96272323 -4.70669256] , -1.9503477816008707 , 20032

[-4.96071309 -4.71010348] , -1.9515227271221454 , 20032

[-1.17494267e+74 -1.17494267e+74] , -0.8118472748889166 , 20032

Giá trị trung bình (mean) của Michalewicz (d=2, n=32) trong CEMv2: -1.2092059142782203
Độ lệch chuẩn (std) của Michalewicz (d=2, n=32) trong CEMv2: 0.5003429394403275


In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=2, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=2, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[2.59363145e+40 2.59363143e+40] , -0.9457514859150447 , 20032

[-3.77259834e+38 -3.77259842e+38] , -1.0324727378183147 , 20032

[-3.13007290e+37 -3.13007301e+37] , -1.0170794224090345 , 20032

[2.78664167e+39 2.78664172e+39] , -1.1350011721457327 , 20032

[2.07575854e+38 2.07575856e+38] , -1.3511881886915766 , 20032

[-6.46077810e+38 -6.46077784e+38] , -0.8987619354554233 , 20032

[-4.96859392 -5.66024184] , -1.5432122278923697 , 20032

[-4.96497524 -4.71263668] , -1.9609452707206252 , 20032

[-3.90644578e+38 -3.90644560e+38] , -0.9875822700279435 , 20032

[-1.67204328e+43 -1.67204328e+43] , -0.7272024482639119 , 20032

Giá trị trung bình (mean) của Michalewicz (d=2, n=64) trong CEMv2: -1.1599197159339973
Độ lệch chuẩn (std) của Michalewicz (d=2, n=64) trong CEMv2: 0.34545934339375994


In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=2, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=2, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-2.26314167e+21 -2.26314177e+21] , -0.8024139850602902 , 20096

[1.27517438e+19 1.27517439e+19] , -0.925191559619201 , 20096

[8.88591098e+20 8.88591095e+20] , -1.0620787743930065 , 20096

[4.58353637e+20 4.58353640e+20] , -0.9742298259776879 , 20096

[-1.54900788e+21 -1.54900790e+21] , -0.9625447565283608 , 20096

[-4.9612686  -4.70986345] , -1.9671741824497087 , 20096

[-4.96458852 -5.20709918] , -1.8463699778927185 , 20096

[-2.17162286e+20 -2.17162289e+20] , -1.0999776199249274 , 20096

[-6.20056477e+20 -6.20056487e+20] , -0.975935141459339 , 20096

[2.66898719e+20 2.66898724e+20] , -0.8799187186631523 , 20096

Giá trị trung bình (mean) của Michalewicz (d=2, n=128) trong CEMv2: -1.1495834541968393
Độ lệch chuẩn (std) của Michalewicz (d=2, n=128) trong CEMv2: 0.3877586110488714


In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=2, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=2, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-3.76534359e+09 -3.76534376e+09] , -0.9991893250065768 , 20224

[-3.85560336 -5.20865615] , -1.5291843508644911 , 20224

[7.50032940e+09 7.50032937e+09] , -1.382603487956473 , 20224

[2.17826842e+10 2.17826843e+10] , -1.667394430422943 , 20224

[8.08784708e+08 8.08784647e+08] , -1.5591111227652297 , 20224

[-5.87158931 -4.71197213] , -1.3962896263420388 , 20224

[-4.96406613 -4.71132276] , -1.967114039636301 , 20224

[5.34255578e+09 5.34255577e+09] , -1.5051357858542884 , 20224

[-4.96635984 -5.20786478] , -1.8466176856595184 , 20224

[2.12100768e+09 2.12100759e+09] , -0.907741083700493 , 20224

Giá trị trung bình (mean) của Michalewicz (d=2, n=256) trong CEMv2: -1.4760380938208355
Độ lệch chuẩn (std) của Michalewicz (d=2, n=256) trong CEMv2: 0.31481907155233124


In [ ]:
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=10, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=10, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-1.22721623e+148 -1.22721624e+148 -1.22721624e+148 -1.22721624e+148
 -1.22721624e+148 -1.22721624e+148 -1.22721624e+148 -1.22721624e+148
 -1.22721624e+148 -1.22721624e+148] , -2.1918200880658336 , 4928

[4.52902014e+147 4.52902001e+147 4.52902001e+147 4.52902001e+147
 4.52902001e+147 4.52902001e+147 4.52902001e+147 4.52902001e+147
 4.52902001e+147 4.52902001e+147] , -4.101454316352045 , 4912

[-6.08445375e+147 -6.08445379e+147 -6.08445379e+147 -6.08445379e+147
 -6.08445379e+147 -6.08445379e+147 -6.08445379e+147 -6.08445379e+147
 -6.08445379e+147 -6.08445379e+147] , -2.1317991295368466 , 4960

[-7.03694431e+147 -7.03694363e+147 -7.03694363e+147 -7.03694363e+147
 -7.03694363e+147 -7.03694363e+147 -7.03694363e+147 -7.03694363e+147
 -7.03694363e+147 -7.03694363e+147] , -2.513337446961109 , 5008

[-1.83957799e+147 -1.83957809e+147 -1.83957809e+147 -1.83957809e+147
 -1.83957809e+147 -1.83957809e+147 -1.83957809e+147 -1.83957809e+147
 -1.83957809e+147 -1.83957809e+147] , -2.432499322067169 ,

In [ ]:
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=10, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=10, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[1.40430338e+148 1.40430340e+148 1.40430340e+148 1.40430340e+148
 1.40430340e+148 1.40430340e+148 1.40430340e+148 1.40430340e+148
 1.40430340e+148 1.40430340e+148] , -2.6734676249142124 , 9920

[5.03670645e+147 5.03670592e+147 5.03670592e+147 5.03670592e+147
 5.03670592e+147 5.03670592e+147 5.03670592e+147 5.03670592e+147
 5.03670592e+147 5.03670592e+147] , -2.964418685306703 , 10048

[7.84181667e+146 7.84181634e+146 7.84181634e+146 7.84181634e+146
 7.84181634e+146 7.84181634e+146 7.84181634e+146 7.84181634e+146
 7.84181634e+146 7.84181634e+146] , -1.922044292093696 , 10016

[3.67848828e+147 3.67848821e+147 3.67848821e+147 3.67848821e+147
 3.67848821e+147 3.67848821e+147 3.67848821e+147 3.67848821e+147
 3.67848821e+147 3.67848821e+147] , -2.0085911813774175 , 9920

[1.70698977e+146 1.70698953e+146 1.70698953e+146 1.70698953e+146
 1.70698953e+146 1.70698953e+146 1.70698953e+146 1.70698953e+146
 1.70698953e+146 1.70698953e+146] , -2.4233346573514583 , 9664

[-6.47934395e+146 -6.47934928e

In [ ]:
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=10, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=10, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-4.24239440e+145 -4.24239463e+145 -4.24239463e+145 -4.24239463e+145
 -4.24239463e+145 -4.24239463e+145 -4.24239463e+145 -4.24239463e+145
 -4.24239463e+145 -4.24239463e+145] , -3.2867305221092815 , 20032

[1.06207390e+147 1.06207396e+147 1.06207396e+147 1.06207396e+147
 1.06207396e+147 1.06207396e+147 1.06207396e+147 1.06207396e+147
 1.06207396e+147 1.06207396e+147] , -3.0840232846277904 , 19520

[-1.67776378e+148 -1.67776380e+148 -1.67776380e+148 -1.67776380e+148
 -1.67776380e+148 -1.67776380e+148 -1.67776380e+148 -1.67776380e+148
 -1.67776380e+148 -1.67776380e+148] , -3.903937394295593 , 19456

[-2.54371535e+147 -2.54371531e+147 -2.54371531e+147 -2.54371531e+147
 -2.54371531e+147 -2.54371531e+147 -2.54371531e+147 -2.54371531e+147
 -2.54371531e+147 -2.54371531e+147] , -2.844330041630702 , 19968

[-5.97995463e+146 -5.97995553e+146 -5.97995553e+146 -5.97995553e+146
 -5.97995553e+146 -5.97995553e+146 -5.97995553e+146 -5.97995553e+146
 -5.97995553e+146 -5.97995553e+146] , -3.0413234328129

In [ ]:
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=10, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=10, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[2.72615332e+147 2.72615330e+147 2.72615330e+147 2.72615330e+147
 2.72615330e+147 2.72615330e+147 2.72615330e+147 2.72615330e+147
 2.72615330e+147 2.72615330e+147] , -4.068470820383514 , 41472

[-6.37133118e+146 -6.37133116e+146 -6.37133116e+146 -6.37133116e+146
 -6.37133116e+146 -6.37133116e+146 -6.37133116e+146 -6.37133116e+146
 -6.37133116e+146 -6.37133116e+146] , -3.32740063518501 , 39680

[1.78249632e+147 1.78249637e+147 1.78249637e+147 1.78249637e+147
 1.78249637e+147 1.78249637e+147 1.78249637e+147 1.78249637e+147
 1.78249637e+147 1.78249637e+147] , -3.2112971722132833 , 40320

[3.89507082e+147 3.89507089e+147 3.89507089e+147 3.89507089e+147
 3.89507089e+147 3.89507089e+147 3.89507089e+147 3.89507089e+147
 3.89507089e+147 3.89507089e+147] , -2.905476965997556 , 39808

[6.23414449e+147 6.23414453e+147 6.23414453e+147 6.23414453e+147
 6.23414453e+147 6.23414453e+147 6.23414453e+147 6.23414453e+147
 6.23414453e+147 6.23414453e+147] , -3.5295767347256994 , 39552

[-4.38812831e+146 -

In [ ]:
d = 10
max_evals = 100000
f =   Michalewicz_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của Michalewicz (d=10, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của Michalewicz (d=10, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-2.11763353e+147 -2.11763355e+147 -2.11763355e+147 -2.11763355e+147
 -2.11763355e+147 -2.11763355e+147 -2.11763355e+147 -2.11763355e+147
 -2.11763355e+147 -2.11763355e+147] , -2.5945299074766206 , 78848

[-5.84194185e+146 -5.84194177e+146 -5.84194177e+146 -5.84194177e+146
 -5.84194177e+146 -5.84194177e+146 -5.84194177e+146 -5.84194177e+146
 -5.84194177e+146 -5.84194177e+146] , -3.190639048826854 , 78336

[-1.95239366e+147 -1.95239368e+147 -1.95239368e+147 -1.95239368e+147
 -1.95239368e+147 -1.95239368e+147 -1.95239368e+147 -1.95239368e+147
 -1.95239368e+147 -1.95239368e+147] , -3.526538389999267 , 79360

[-1.31151677e+146 -1.31151660e+146 -1.31151660e+146 -1.31151660e+146
 -1.31151660e+146 -1.31151660e+146 -1.31151660e+146 -1.31151660e+146
 -1.31151660e+146 -1.31151660e+146] , -4.064437286805737 , 79872

[8.79014804e+146 8.79014786e+146 8.79014786e+146 8.79014786e+146
 8.79014786e+146 8.79014786e+146 8.79014786e+146 8.79014786e+146
 8.79014786e+146 8.79014786e+146] , -3.31788957556546

 **Ackley**

In [ ]:
d = 2
max_evals = 20000
f =  Ackley_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[0.93955798 0.00466171] , 2.5872554599412223 , 20016

[-0.02364866 -0.03018311] , 0.12146200961603215 , 20016

[-0.97708624  0.97954165] , 3.586713959121728 , 20016

[-0.00541346 -0.00808838] , 0.06138159425246714 , 20016

[-0.95898232  0.0073561 ] , 2.592109891462734 , 20016

[-0.00693177 -0.00492812] , 0.014744647382720633 , 20016

[-0.01751923  0.0069897 ] , 0.0547890416329051 , 20016

[-0.01035289 -0.00227948] , 0.03374772238614954 , 20016

[ 0.00473892 -0.01745495] , 0.021694530774891252 , 20016

[ 0.00988427 -0.0188844 ] , 0.07280073188657754 , 20016

Giá trị trung bình (mean) của  Ackley (d=2, n=16) trong CEMv2: 0.9146699588457426
Độ lệch chuẩn (std) của  Ackley (d=2, n=16) trong CEMv2: 1.339398359877374


In [ ]:
d = 2
max_evals = 20000
f =  Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.00171617  0.00763817] , 0.030539600670501432 , 20032

[ 0.00445348 -0.00173696] , 0.03895202053901814 , 20032

[0.00268705 0.00761195] , 0.026218592393924656 , 20032

[0.95134631 0.00284952] , 2.5806405146243105 , 20032

[-0.96731824  0.97467718] , 3.583303232182488 , 20032

[0.00016748 0.00381536] , 0.01673556471737614 , 20032

[ 0.0015337  -0.00628915] , 0.010365088851969961 , 20032

[-0.003871    0.00630429] , 0.02201541780661076 , 20032

[-5.96371902e-05  1.40006998e-03] , 0.04766760403228831 , 20032

[0.0009768  0.95392244] , 2.5842256923256284 , 20032

Giá trị trung bình (mean) của  Ackley (d=2, n=32) trong CEMv2: 0.8940663328144115
Độ lệch chuẩn (std) của  Ackley (d=2, n=32) trong CEMv2: 1.3487299807089876


In [ ]:
d = 2
max_evals = 20000
f =  Ackley_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-7.78685780e-04 -9.50700992e-01] , 2.5834546745819336 , 20032

[ 9.53617012e-01 -5.88523995e-05] , 2.5834471666715797 , 20032

[0.005223   0.95536146] , 2.580295807673487 , 20032

[ 0.00049785 -0.00074366] , 0.003907092152943381 , 20032

[0.0035191  0.94906298] , 2.5822097290431647 , 20032

[ 0.00193414 -0.0020038 ] , 0.01047785450775196 , 20032

[-0.00220932  0.00098316] , 0.01493496329813171 , 20032

[-0.00304019  0.0076426 ] , 0.017920561864788187 , 20032

[-0.00164389 -0.9457921 ] , 2.5810517043642176 , 20032

[0.00109075 0.00849643] , 0.005994592792710751 , 20032

Giá trị trung bình (mean) của  Ackley (d=2, n=64) trong CEMv2: 1.2963694146950708
Độ lệch chuẩn (std) của  Ackley (d=2, n=64) trong CEMv2: 1.2857280953262018


In [ ]:
d = 2
max_evals = 20000
f =  Ackley_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.00098439  0.00138231] , 0.009950662048411107 , 20096

[9.54146331e-01 4.18407543e-04] , 2.5822927959984807 , 20096

[-0.47854122  0.47645245] , 2.581688667625603 , 20096

[-0.00290641 -0.00266189] , 0.007488169758041341 , 20096

[0.00307013 0.00395752] , 0.015948112718224383 , 20096

[0.00172067 0.00225374] , 0.014489382447564925 , 20096

[-0.00021472 -0.00181221] , 0.004518592245756015 , 20096

[-0.00027783  0.00319246] , 0.01319166461015131 , 20096

[-0.00256436 -0.94722015] , 2.580706741135057 , 20096

[ 0.96701063 -0.97155705] , 3.5747700745159894 , 20096

Giá trị trung bình (mean) của  Ackley (d=2, n=128) trong CEMv2: 1.1385044863103277
Độ lệch chuẩn (std) của  Ackley (d=2, n=128) trong CEMv2: 1.4075251325385005


In [ ]:
d = 2
max_evals = 20000
f =  Ackley_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=2, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=2, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-0.0001638   0.00092547] , 0.00660907811567002 , 20224

[-0.00099556 -0.00083148] , 0.010175001266674588 , 20224

[-0.96632553  0.97090184] , 3.5747849085333425 , 20224

[8.95000780e-04 2.64275966e-05] , 0.0031818159140653 , 20224

[ 0.373366   -0.57331301] , 2.5935101164578396 , 20224

[-0.00028632  0.00069827] , 0.009392048062569547 , 20224

[-0.0011552   0.00210371] , 0.0024913311071022015 , 20224

[0.00049708 0.00216701] , 0.006963746656492464 , 20224

[-0.00027617  0.00306908] , 0.005227661872588829 , 20224

[ 0.9687455  -0.96750588] , 3.574788203707936 , 20224

Giá trị trung bình (mean) của  Ackley (d=2, n=256) trong CEMv2: 0.978712391169428
Độ lệch chuẩn (std) của  Ackley (d=2, n=256) trong CEMv2: 1.5068523715202158


In [ ]:
d = 10
max_evals = 100000
f =  Ackley_f
seed_number = 21522678
N = 16 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=16) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=16) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[2.05131232e+147 2.05131210e+147 2.05131210e+147 2.05131210e+147
 2.05131210e+147 2.05131210e+147 2.05131210e+147 2.05131210e+147
 2.05131210e+147 2.05131210e+147] , 20.026895454367367 , 4864

[-7.48581034e+147 -7.48581028e+147 -7.48581028e+147 -7.48581028e+147
 -7.48581028e+147 -7.48581028e+147 -7.48581028e+147 -7.48581028e+147
 -7.48581028e+147 -7.48581028e+147] , 20.379552291057767 , 4944

[6.62102879e+147 6.62102894e+147 6.62102894e+147 6.62102894e+147
 6.62102894e+147 6.62102894e+147 6.62102894e+147 6.62102894e+147
 6.62102894e+147 6.62102894e+147] , 21.37635825368575 , 4992

[-1.97042153e+146 -1.97042215e+146 -1.97042215e+146 -1.97042215e+146
 -1.97042215e+146 -1.97042215e+146 -1.97042215e+146 -1.97042215e+146
 -1.97042215e+146 -1.97042215e+146] , 20.742400594201545 , 5008

[-2.22449290e+147 -2.22449285e+147 -2.22449285e+147 -2.22449285e+147
 -2.22449285e+147 -2.22449285e+147 -2.22449285e+147 -2.22449285e+147
 -2.22449285e+147 -2.22449285e+147] , 20.282065210143685 , 5008

[4.743

In [ ]:
d = 10
max_evals = 100000
f =  Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=32) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=32) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 0.06975223 -0.01725244  0.03149781  0.05170702 -0.00867656  0.04778767
 -0.03243279 -0.93497495  0.76964771 -0.03737581] , 1.7487730631375062 , 100032

[ 0.50158605 -0.32675583  0.58691677 -0.09761551 -0.00649014 -0.19273566
 -0.26649529 -0.23454827 -0.27287054  0.81181015] , 2.564940250417312 , 100032

[ 0.07561666  0.01632135 -0.88913785  0.0486677   0.01354357 -0.04196245
  0.06902716  0.89717754  0.94537914  0.01319991] , 2.0971909113084988 , 100032

[-0.03307818 -0.04493652 -0.87845897 -0.01187831  0.03168058  0.0070037
 -0.91061583 -0.03151511 -0.95257744 -0.04335842] , 2.076176785449974 , 100032

[ 0.00648147  0.03155689 -0.01995853 -0.03357717  0.86519795  0.03648523
  0.07463798 -0.03827034 -0.008021   -0.59520327] , 1.8547006215208026 , 100032

[-1.00046581  0.05730102 -0.9812715   1.02509881  0.08085518 -0.91958868
  0.90940638 -0.00794264 -0.99300797  0.9424977 ] , 3.1583917715996432 , 100032

[-0.04622848  0.06833055 -0.91436354  0.03607484 -0.09176332 -0.07075019
  0.90

In [ ]:
d = 10
max_evals = 100000
f =  Ackley_f
seed_number = 21522678
N = 64 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=64) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=64) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-9.13049942e-01  5.72922886e-03  9.14465583e-01  6.00498414e-03
  2.78594453e-03  1.17456622e-02 -1.21100632e-02  8.21758215e-04
 -8.57659355e-03  1.63683774e-02] , 1.6744987220374665 , 100032

[ 0.95891536 -0.00621435  0.92666337  0.96098056 -0.0114111  -0.00581568
 -0.92681353 -0.95315823 -0.00539781  0.01052847] , 2.6110651647319156 , 100032

[-0.92313292  0.00833739  0.0054876   0.93856176  0.00437144 -0.95361614
 -0.93101153  0.00266825 -0.01406192 -0.0112855 ] , 2.3512750704477736 , 100032

[-0.00661742 -0.01132122 -0.03182608 -0.0061177   0.92757245 -0.90491672
 -0.00431593  0.91768922 -0.00552687  0.0071553 ] , 2.034590330624767 , 100032

[ 0.94804629 -0.00201658 -0.92926055 -0.95963401  0.94857633  0.92896746
 -0.00111732  0.0137995  -0.00405292  0.95944991] , 2.8467180545703887 , 100032

[-0.00943176 -0.0055005  -0.89703539  0.00116521 -0.00318827 -0.01214871
 -0.00890117 -0.91574627 -0.01285261  0.01467607] , 1.6783297193429552 , 100032

[ 0.00611438  0.962682    0.00503939

In [ ]:
d = 10
max_evals = 100000
f =  Ackley_f
seed_number = 21522678
N = 128 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=128) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=128) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[ 5.82168125e-03  1.28061487e-02  8.58394747e-01 -5.26127407e-03
  5.75283456e-03  3.27321108e-03 -7.62237876e-04 -8.66748079e-03
 -1.84108121e-03  1.09653000e-02] , 1.1876423717351838 , 100096

[-5.67781377e-04 -9.17326624e-01 -2.08262722e-02 -1.25501467e-02
  1.58373506e-02 -5.00499218e-03 -9.26874620e-01 -9.42412348e-01
 -6.08746140e-03 -9.00779106e-03] , 2.0392916045770844 , 100096

[ 1.38456639e-02  9.41537736e-01 -1.58488708e-02  1.03592850e-02
  6.19676248e-03  7.48773765e-03 -9.12035493e-01  3.45930966e-04
  9.26903469e-01 -6.84970091e-03] , 2.029996952024885 , 100096

[-0.9599385  -0.00516054 -0.01268923 -0.94286657 -0.00849218 -0.00176961
 -0.0130193   0.94286242  0.01303749 -0.94180639] , 2.337612782822188 , 100096

[-0.01513931  0.01569364 -0.01868247 -0.95456906  1.90915488  0.94941333
 -0.00916657  0.00225087  0.00340352  0.01285171] , 2.8333484284029464 , 100096

[-2.46700873e-02 -8.95030781e-01 -9.23877964e-01 -1.27981947e-02
 -1.37095720e-02  4.12548784e-03 -5.67262387

In [ ]:
d = 10
max_evals = 100000
f =  Ackley_f
seed_number = 21522678
N = 256 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01

all_results = []
all_fitness = []
num_evals = []

for i in range(10):
    results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
    all_results.append(results[-1])
    all_fitness.append(fitness[-1])
    num_evals.append(evals[-1])
    print(str(results[-1])+' , '+str(fitness[-1])+' , '+ str(evals[-1]))
    print()
    seed_number += 1
print(f"Giá trị trung bình (mean) của  Ackley (d=10, n=256) trong CEMv2: {np.mean([all_fitness[i] for i in range(len(all_results))])}")
print(f"Độ lệch chuẩn (std) của  Ackley (d=10, n=256) trong CEMv2: {np.std([all_fitness[i] for i in range(len(all_results))])}")

[-1.09087013e-02 -9.98384896e-05 -1.17636701e-02 -5.22669604e-03
  6.21415660e-03  4.00557199e-03 -1.80369109e-02  6.04467309e-03
 -1.05324729e-02 -7.04932112e-03] , 0.07812593714340865 , 100096

[-5.91990556e-03 -9.15690061e-01  4.23301413e-03 -4.10431278e-03
 -8.27276299e-03 -1.77315687e-03 -2.30312678e-02  7.25822373e-04
  4.01137492e-03  9.19369182e-01] , 1.663914026372805 , 100096

[ 0.96695142  0.94569939 -0.9427334   0.95739736 -0.00259761  0.00962106
  0.00135797 -0.94545948 -0.00257675  0.0130931 ] , 2.5907979154834817 , 100096

[-0.01290531 -0.00412007  0.00093517 -0.00335778  0.00462714  0.00168992
  0.01991441  0.92626415  0.90562538  0.00892715] , 1.662993822855349 , 100096

[ 9.48997265e-01  4.69660232e-04 -2.77613698e-03  1.34955445e-03
  9.55111819e-01 -5.52143591e-03  6.33401288e-03 -7.67964888e-03
  9.35736095e-01 -9.59881245e-01] , 2.3328446768536932 , 100096

[-0.00378107  0.00905393 -0.00258169  0.00762622 -0.00180318  0.01662499
 -0.00513402 -0.0160315   0.8692383